#***Data Mining Project  1402-1***

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import shapiro
#Disable unwanted warnings
import warnings
warnings.filterwarnings("ignore")

#Mounting mydrive & Importing data ***(describing and preprocessing)***

In [ ]:
df = pd.read_csv("/Users/mahdimacbook/Downloads/Airline.csv")

In [ ]:
df

##Get familiar with df

In [ ]:
df.tail()

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.describe()

Fortunately, there were **no abnormal Ranges** for each columns (unusual min and max) in df. If there were unusual ranges, we could use following code:
* **df.replace([np.inf, -np.inf], np.nan, inplace=True)**





##Preprocessing

In [ ]:
# Check for missing values in each column
print(df.isnull().sum())

# Visualize missing values using a heatmap
import seaborn as sns
sns.heatmap(df.isnull(), cbar=False, cmap='viridis')

In [ ]:
df[df.duplicated()]

In [ ]:
#due to high number of records, we can omit 393 null rows (393/129880 is equal to less than 0.3% !).
df = df.dropna(axis=0)
df.shape

In [ ]:
#check it
print(df.shape,'\n-----------------')
print(df.count(),'\n----------------')
print(df.isna().any(axis=1).sum())

In [ ]:
df.columns

In [ ]:
for i in df:
  print(df[i].unique(),'\n------------------------------')

Correcting String Values:

In [ ]:
df['Type of Travel'].replace('Business travel', 'Business Travel', inplace=True)

df['Customer Type'].replace('disloyal Customer', 'Disloyal Customer', inplace=True)

df['satisfaction'].replace('disloyal Customer', 'Disloyal Customer', inplace=True)

**Outlier Detection Methods:**

* Z-Score (Standard Score) Method: This method measures how many standard deviations a data point is from the mean. Data points with a high absolute Z-score are considered outliers.

* IQR (Interquartile Range) Method: Calculate the IQR, which is the range between the first quartile (Q1) and the third quartile (Q3). Data points outside the range [Q1 - 1.5 * IQR, Q3 + 1.5 * IQR] are considered outliers.

* Visual Inspection: Plotting the data using box plots, histograms, or scatter plots can help identify outliers by visually inspecting the data distribution.

* Machine Learning Algorithms: Some machine learning algorithms, such as isolation forests and one-class SVM, can be used to detect outliers by modeling the inliers and identifying data points that don't fit the model.

* Distance-Based Methods: Calculate distances between data points and look for data points with significantly larger distances from their neighbors.

* Statistical Tests: Use statistical tests like the Grubbs' test or the Dixon's test to identify outliers based on a specific significance level.

**Outlier Handling Methods:**

* Removal: The simplest approach is to remove the outlier data points from the dataset. This can be effective when the outliers are rare and don't carry valuable information.

* Capping/Flooring: Capping sets a maximum and minimum threshold for outlier values. Outliers beyond these thresholds are replaced with the threshold values.

* Transformation: Apply mathematical transformations to the data, such as log transformations, to reduce the impact of outliers.

* Imputation: Replace outliers with missing values or impute them using techniques like mean, median, or regression imputation.

* Winsorization: Instead of removing or capping outliers, Winsorization reduces the impact of outliers by replacing them with values near the threshold.

* Model-Based Correction: Train a statistical model or machine learning model to predict the correct values for outliers based on the rest of the data.

* Data Segmentation: Split the data into segments or clusters and handle outliers separately within each segment.

**The choice of an outlier detection and handling method for  DataFrame (df) depends on the specific characteristics of  data, the goals of  analysis, and the potential impact on the quality of  results. Two common methods:**

* Z-Score Method: The Z-score method is suitable for data that follows a normal distribution or is approximately normally distributed. It measures how many standard deviations a data point is from the mean. Data points with high absolute Z-scores (e.g., greater than 3) are considered outliers. The advantage of this method is its simplicity and ease of implementation. However, it may not work well for data that is not normally distributed.

* IQR (Interquartile Range) Method: The IQR method is robust and can be applied to data with various distributions, including skewed data. It defines outliers based on the range between the first quartile (Q1) and the third quartile (Q3). Data points outside the range [Q1 - 1.5 * IQR, Q3 + 1.5 * IQR] are considered outliers. This method is less sensitive to extreme values than the Z-score method and can handle skewed data effectively.



###Outlier detection using   :

**AGE**

In [ ]:
import matplotlib.pyplot as plt

# Creating plot
plt.boxplot(df['Age'])

# show plot
plt.show()

--------------------------------------------------------------------------------------------
**DISTANCE**

In [ ]:
# Creating plot
plt.boxplot(df['Flight Distance'])

# show plot
plt.show()

In [ ]:
x = df['Flight Distance']
plt.hist(x, bins=60 )
plt.show()

It is clear from plot that distribution isn't Normal! but we have to do some statistical tests to prove this fact.

In [ ]:
plt.figure(figsize=(16,5))
sns.distplot(df['Flight Distance'])
plt.show()

In [ ]:
sns.boxplot(df['Flight Distance'])

Is Flight distances following a **NORMAL distribution**?

In [ ]:
# Plot a histogram to visualize the distribution
plt.figure(figsize=(8, 6))
sns.histplot(df['Flight Distance'], kde=True)
plt.title('Flight Distance Histogram', fontsize=14)
plt.xlabel('Flight Distance', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.show()

# Perform the Shapiro-Wilk normality test
statistic, p_value = shapiro(df['Flight Distance'])

# Set the significance level (alpha)
alpha = 0.05

# Check the p-value against alpha
if p_value > alpha:
    print(f"The Flight Distance column appears to be normally distributed (p-value={p_value:.4f})")
else:
    print(f"The Flight Distance column does not appear to be normally distributed (p-value={p_value:.4f})")

So after all, we have to use IQR.


Age **IQR Based Filtering**

In [ ]:
#Finding the IQR
percentile25 = df['Flight Distance'].quantile(0.25)
percentile75 = df['Flight Distance'].quantile(0.75)

#Finding the upper and lower limits
iqr = percentile75 - percentile25
upper_limit = percentile75 + 1.5 * iqr
lower_limit = percentile25 - 1.5 * iqr

#Finding outliers
df[df['Flight Distance'] > upper_limit]
df[df['Flight Distance'] < lower_limit]

#Trimming outliers
new_df = df[df['Flight Distance'] < upper_limit]

print(new_df.shape,upper_limit)

There are 4319 outliers based on IQR Based Filtering.

In [ ]:
sns.boxplot(new_df['Flight Distance'])

The outliers are just irregular compared to the other data points, but can happen, take care of them! So we have to utilize CAPPING.

In [ ]:
#Capping
new_df_cap = df.copy()
new_df_cap['Flight Distance'] = np.where(
    new_df_cap['Flight Distance'] > upper_limit,
    upper_limit,
    np.where(
        new_df_cap['Flight Distance'] < lower_limit,
        lower_limit,
        new_df_cap['Flight Distance']
    )
)

In [ ]:
# Create a new figure with adjusted subplot spacing
fig, axes = plt.subplots(1, 3, figsize=(18, 5), sharey=True, gridspec_kw={'wspace': 0.05})


sns.boxplot(df['Flight Distance'], ax=axes[0])
axes[0].set_title('Box Plot of Flight Distance (Original)', fontsize=10)  # Adjust the font size


sns.boxplot(new_df['Flight Distance'], ax=axes[1])
axes[1].set_title('Box Plot of Flight Distance (omitted Upper 1.5*IQR)', fontsize=10)  # Adjust the font size


sns.boxplot(new_df_cap['Flight Distance'], ax=axes[2])
axes[2].set_title('Box Plot of Flight Distance (omitted Upper 1.5*IQR and Capped)', fontsize=10)  # Adjust the font size

plt.show()

-----------------------------------------------------------------------------------------
**DEPARTURE & ARRIVAL DELAY**

In [ ]:
# Creating plot
plt.boxplot([df['Departure Delay in Minutes'],df['Arrival Delay in Minutes']])

# show plot
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Extract the data for the scatter plot
x_departure_delay = df['Departure Delay in Minutes']

# Create a scatter plot for "Departure Delay in Minutes"
plt.figure(figsize=(10, 6))
plt.scatter(x_departure_delay, range(len(x_departure_delay)), alpha=0.5, c='b', label='Departure Delay')
plt.xlabel('Departure Delay in Minutes')
plt.ylabel('Index')
plt.title('Scatter Plot of Departure Delay in Minutes')
plt.grid(True)
plt.legend()
plt.show()


In [ ]:
import matplotlib.pyplot as plt



# Extract the data for the scatter plot
x_Arrival_delay = df['Arrival Delay in Minutes']

# Create a scatter plot for "Arrival Delay in Minutes"
plt.figure(figsize=(10, 6))
plt.scatter(x_Arrival_delay, range(len(x_Arrival_delay)), alpha=0.5, c='r', label='Arrival Delay')
plt.xlabel('Arrival Delay in Minutes')
plt.ylabel('Index')
plt.title('Scatter Plot of Arrival Delay in Minutes')
plt.grid(True)
plt.legend()
plt.show()


Pair plot

In [ ]:
selected_items = ['Arrival Delay in Minutes', 'Departure Delay in Minutes', 'Flight Distance', 'Age']

# Filter the DataFrame for selected items
selected_df = df[selected_items + ['satisfaction']]

# Create a pair plot
pair_plot = sns.pairplot(selected_df, hue='satisfaction', palette={'satisfied': 'green', 'dissatisfied': 'red'})

# Add a legend
pair_plot.add_legend()

# Show the plot
plt.show()

In [ ]:
x = df['Departure Delay in Minutes']
plt.hist(x, bins=60 )
plt.show()

In [ ]:
x = df['Arrival Delay in Minutes']
plt.hist(x, bins=60 )
plt.show()

They don't appear to be normally distributed, but we have to prove it.

Are  "Arrival Delay in Minutes" and "Departure Delay in Minutes"  following **NORMAL distribution**?

In [ ]:
# Columns to test
columns_to_test = ['Arrival Delay in Minutes', 'Departure Delay in Minutes']

for column in columns_to_test:
    # Plot a histogram to visualize the distribution
    plt.figure(figsize=(6, 4))
    sns.histplot(df[column], kde=True)
    plt.title(f'{column} Histogram', fontsize=14)
    plt.xlabel(column, fontsize=12)
    plt.ylabel('Frequency', fontsize=12)
    plt.show()

    # Perform the Shapiro-Wilk normality test
    statistic, p_value = shapiro(df[column])

    # Set the significance level (alpha)
    alpha = 0.05

    # Check the p-value against alpha
    if p_value > alpha:
        print(f"The {column} column appears to be normally distributed (p-value={p_value:.4f})")
    else:
        print(f"The {column} column does not appear to be normally distributed (p-value={p_value:.4f})")

Arrival Delay in Minutes **IQR Based Filtering**

In [ ]:
#Finding the IQR
percentile25 = df['Arrival Delay in Minutes'].quantile(0.25)
percentile75 = df['Arrival Delay in Minutes'].quantile(0.75)

#Finding the upper and lower limits
iqr = percentile75 - percentile25
upper_limit = percentile75 + 1.5 * iqr
lower_limit = percentile25 - 1.5 * iqr

#Finding outliers
df[df['Arrival Delay in Minutes'] > upper_limit]
df[df['Arrival Delay in Minutes'] < lower_limit]

#Trimming outliers
new_df = df[df['Arrival Delay in Minutes'] < upper_limit]

print(new_df.shape,upper_limit)

There are 17492 outliers based on IQR Based Filtering.  

In [ ]:
#Capping
new_df_cap = df.copy()
new_df_cap['Arrival Delay in Minutes'] = np.where(
    new_df_cap['Arrival Delay in Minutes'] > upper_limit,
    upper_limit,
    np.where(
        new_df_cap['Arrival Delay in Minutes'] < lower_limit,
        lower_limit,
        new_df_cap['Arrival Delay in Minutes']
    )
)
new_df_cap.shape

In [ ]:
# Create a new figure with adjusted subplot spacing
fig, axes = plt.subplots(1, 3, figsize=(18, 5), sharey=True, gridspec_kw={'wspace': 0.1})


sns.boxplot(df['Arrival Delay in Minutes'], ax=axes[0])
axes[0].set_title('Box Plot of Arrival Delay in Minutes (Original)', fontsize=10)  # Adjust the font size


sns.boxplot(new_df['Arrival Delay in Minutes'], ax=axes[1])
axes[1].set_title('Box Plot of Arrival Delay in Minutes (omitted Upper 1.5*IQR)', fontsize=10)  # Adjust the font size


sns.boxplot(new_df_cap['Arrival Delay in Minutes'], ax=axes[2])
axes[2].set_title('Box Plot of Arrival Delay in Minutes (omitted Upper 1.5*IQR and Capped)', fontsize=10)  # Adjust the font size

plt.show()

In [ ]:
sns.boxplot(new_df['Arrival Delay in Minutes'])

In [ ]:
sns.boxplot(new_df_cap['Arrival Delay in Minutes'])

Departure Delay in Minutes **IQR Based Filtering**

In [ ]:
#Finding the IQR
percentile25 = df['Departure Delay in Minutes'].quantile(0.25)
percentile75 = df['Departure Delay in Minutes'].quantile(0.75)

#Finding the upper and lower limits
iqr = percentile75 - percentile25
upper_limit = percentile75 + 1.5 * iqr
lower_limit = percentile25 - 1.5 * iqr

#Finding outliers
df[df['Departure Delay in Minutes'] > upper_limit]
df[df['Departure Delay in Minutes'] < lower_limit]

#Trimming outliers
new_df = df[df['Departure Delay in Minutes'] < upper_limit]

print(new_df.shape,upper_limit)

There are 18512 outliers based on IQR Based Filtering.

In [ ]:
#Capping
new_df_cap = df.copy()
new_df_cap['Departure Delay in Minutes'] = np.where(
    new_df_cap['Departure Delay in Minutes'] > upper_limit,
    upper_limit,
    np.where(
        new_df_cap['Departure Delay in Minutes'] < lower_limit,
        lower_limit,
        new_df_cap['Departure Delay in Minutes']
    )
)
new_df_cap.shape

In [ ]:
# Create a new figure with adjusted subplot spacing
fig, axes = plt.subplots(1, 3, figsize=(18, 5), sharey=True, gridspec_kw={'wspace': 0.1})


sns.boxplot(df['Departure Delay in Minutes'], ax=axes[0])
axes[0].set_title('Box Plot of Departure Delay in Minutes (Original)', fontsize=10)  # Adjust the font size


sns.boxplot(new_df['Departure Delay in Minutes'], ax=axes[1])
axes[1].set_title('Box Plot of Departure Delay in Minutes (omitted Upper 1.5*IQR)', fontsize=10)  # Adjust the font size


sns.boxplot(new_df_cap['Departure Delay in Minutes'], ax=axes[2])
axes[2].set_title('Box Plot of Departure Delay in Minutes (omitted Upper 1.5*IQR and Capped)', fontsize=10)  # Adjust the font size

plt.show()

In [ ]:
sns.boxplot(new_df['Departure Delay in Minutes'])

In [ ]:
sns.boxplot(new_df_cap['Departure Delay in Minutes'])

**Applying all methods on Original df:**

In [ ]:
#Finding the IQR
percentile25 = df['Flight Distance'].quantile(0.25)
percentile75 = df['Flight Distance'].quantile(0.75)

#Finding the upper and lower limits
iqr = percentile75 - percentile25
upper_limit = percentile75 + 1.5 * iqr
lower_limit = percentile25 - 1.5 * iqr

#Capping
df['Flight Distance'] = np.where(
    df['Flight Distance'] > upper_limit,
    upper_limit,
    np.where(
        df['Flight Distance'] < lower_limit,
        lower_limit,
        df['Flight Distance']
    )
)






#Finding the IQR
percentile25 = df['Arrival Delay in Minutes'].quantile(0.25)
percentile75 = df['Arrival Delay in Minutes'].quantile(0.75)

#Finding the upper and lower limits
iqr = percentile75 - percentile25
upper_limit = percentile75 + 1.5 * iqr
lower_limit = percentile25 - 1.5 * iqr

#Capping
df['Arrival Delay in Minutes'] = np.where(
    df['Arrival Delay in Minutes'] > upper_limit,
    upper_limit,
    np.where(
        df['Arrival Delay in Minutes'] < lower_limit,
        lower_limit,
        df['Arrival Delay in Minutes']
    )
)






#Finding the IQR
percentile25 = df['Departure Delay in Minutes'].quantile(0.25)
percentile75 = df['Departure Delay in Minutes'].quantile(0.75)

#Finding the upper and lower limits
iqr = percentile75 - percentile25
upper_limit = percentile75 + 1.5 * iqr
lower_limit = percentile25 - 1.5 * iqr

#Capping
df['Departure Delay in Minutes'] = np.where(
    df['Departure Delay in Minutes'] > upper_limit,
    upper_limit,
    np.where(
        df['Departure Delay in Minutes'] < lower_limit,
        lower_limit,
        df['Departure Delay in Minutes']
    )
)

In [ ]:
df.describe()

In [ ]:
df

In [ ]:
dec = pd.DataFrame()
dec['decimals'] = df['Flight Distance'].map(lambda x: str(x).split('.')[1]).apply(lambda x: x)
dec['decimals'].max()

All the numbers in Flight Distance are integer!

In [ ]:
df['Flight Distance'] = df['Flight Distance'].astype('int64')

In [ ]:
df.info()

##Data Visualization

In [ ]:
# Set the style for Seaborn
sns.set(style="whitegrid")

# Iterate through each column in the DataFrame
for column in df.columns:
    # Create a new figure for each column
    plt.figure(figsize=(10, 6))

    # Choose the appropriate plot based on the data type of the column
    if pd.api.types.is_numeric_dtype(df[column]):
        # For numeric columns, create a histogram
        sns.histplot(df[column], kde=True)
        plt.title(f'Histogram of {column}', fontsize=16)
        plt.xlabel(column, fontsize=14)
        plt.ylabel('Frequency', fontsize=14)
    else:
        # For categorical columns, create a countplot
        sns.countplot(x=column, data=df, palette='viridis')
        plt.title(f'Countplot of {column}', fontsize=16)
        plt.xlabel(column, fontsize=14)
        plt.ylabel('Count', fontsize=14)

    # Show the plot for the current column
    plt.show()

In [ ]:
def plot_pie_chart(df, column, chart_size=(8, 8), label_font_size=12):
    """
    Create a pie chart for a categorical column in a DataFrame.

    Parameters:
    - df: DataFrame
    - column: str, the column for which to create the pie chart
    """

    # Check if the column exists in the DataFrame
    if column not in df.columns:
        print(f"Column '{column}' not found in the DataFrame.")
        return

    # Get value counts for the column
    value_counts = df[column].value_counts()

    # Create a pie chart with autopct for auto-formatting
    plt.figure(figsize=chart_size)
    plt.pie(value_counts, labels=value_counts.index, autopct='%1.1f%%', startangle=45,
            textprops={'fontsize': label_font_size}, colors=sns.color_palette('viridis'))
    plt.title(f'Pie Chart for {column}', fontsize=12)
    plt.show()


#----------------------------------------------------------------------------

def plot_bar_chart(df, column):
    """
    Create a bar chart for a categorical column in a DataFrame.

    Parameters:
    - df: DataFrame
    - column: str, the column for which to create the bar chart
    """

    # Get value counts for the column
    value_counts = df[column].value_counts()

    # Create a bar chart
    plt.figure(figsize=(12, 6))
    sns.barplot(x=value_counts.index, y=value_counts, palette='viridis')
    plt.title(f'Bar Chart for {column}', fontsize=16)
    plt.xlabel(column, fontsize=14)
    plt.ylabel('Count', fontsize=14)
    plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels for better visibility
    plt.show()

#------------------------------------------------------------------------------

def plot_scatter_chart(df, x_column, y_column):
    """
    Create a scatter plot for two numerical columns in a DataFrame.

    Parameters:
    - df: DataFrame
    - x_column: str, the column for the x-axis
    - y_column: str, the column for the y-axis
    """

    # Create a scatter plot
    plt.figure(figsize=(10, 6))
    sns.scatterplot(x=df[x_column], y=df[y_column], color='blue', alpha=0.7)
    plt.title(f'Scatter Plot: {x_column} vs {y_column}', fontsize=16)
    plt.xlabel(x_column, fontsize=14)
    plt.ylabel(y_column, fontsize=14)
    plt.show()

#-----------------------------------------------------------------------------

def plot_line_chart(df, x_column, y_column):
    """
    Create a line plot for a numerical column in a DataFrame.

    Parameters:
    - df: DataFrame
    - x_column: str, the column for the x-axis
    - y_column: str, the column for the y-axis
    """

    # Create a line plot
    plt.figure(figsize=(10, 6))
    sns.lineplot(x=df[x_column], y=df[y_column], color='green')
    plt.title(f'Line Plot: {y_column} over {x_column}', fontsize=16)
    plt.xlabel(x_column, fontsize=14)
    plt.ylabel(y_column, fontsize=14)
    plt.show()

#------------------------------------------------------------------------------

def plot_histogram(df, column):
    """
    Create a histogram plot for a numerical column in a DataFrame.

    Parameters:
    - df: DataFrame
    - column: str, the column for the histogram
    """

    # Create a histogram plot
    plt.figure(figsize=(10, 6))
    sns.histplot(df[column], kde=True, color='skyblue')
    plt.title(f'Histogram Plot: {column}', fontsize=16)
    plt.xlabel(column, fontsize=14)
    plt.ylabel('Frequency', fontsize=14)
    plt.show()

#------------------------------------------------------------------------------

def plot_violin_plot(dataframe, x_col, y_col):
    """
    Creates a violin plot to visualize the distribution of a numerical variable across different categories.

    A violin plot is a hybrid of a box plot and a kernel density plot, which shows peaks in the data. It is
    used to visualize the distribution of numerical data. Unlike a box plot that can only show summary
    statistics, violin plots depict summary statistics and the density of each variable.

    Parameters:
    - dataframe: pandas DataFrame
    - x_col: Name of the categorical variable (column on the x-axis)
    - y_col: Name of the numerical variable (column being visualized)

    Returns:
    - None (displays the violin plot)
    """
    sns.violinplot(x=x_col, y=y_col, data=dataframe)
    plt.title(f'Violin Plot of {y_col} across {x_col}')
    plt.show()

#------------------------------------------------------------------------------

def plot_grouped_bar(dataframe, x_col, y_col, hue_col):
    """
    Creates a grouped bar plot to show the relationship between two categorical variables.

    Parameters:
    - dataframe: pandas DataFrame
    - x_col: Name of the first categorical variable (x-axis)
    - y_col: Name of the numerical variable (y-axis)
    - hue_col: Name of the second categorical variable for grouping

    Returns:
    - None (displays the grouped bar plot)
    """
    sns.barplot(x=x_col, y=y_col, hue=hue_col, data=dataframe)
    plt.title(f'Grouped Bar Plot of {y_col} across {x_col} grouped by {hue_col}')
    plt.show()


In [ ]:
plot_pie_chart(df, 'satisfaction', chart_size=(4, 4), label_font_size=11)

In [ ]:
# Creating age groups
bins = [0, 18, 30, 50, 70, 90]  # Define  age group bins
labels = ['Younge', 'YoungeAdult', 'Adult', 'MiddleAge', 'Elderly']  # Labels for the age groups

df['Age Group'] = pd.cut(df['Age'], bins=bins, labels=labels, right=False)

plot_pie_chart(df, 'Age Group', chart_size=(5, 5), label_font_size=12)

In [ ]:
plot_bar_chart(df,'Class')

In [ ]:
plot_bar_chart(df,'Customer Type')

In [ ]:
plot_histogram(df,'Age Group')

In [ ]:
plot_violin_plot(df,'Age','satisfaction')

In [ ]:
plot_violin_plot(df,'Food and drink','Seat comfort')

In [ ]:
plot_violin_plot(df,'Seat comfort','satisfaction')

In [ ]:
plot_violin_plot(df,'Flight Distance','Class')

In [ ]:
plot_violin_plot(df,'Leg room service','Age')

In [ ]:
plot_violin_plot(df,'Leg room service','Age')

In [ ]:
plot_grouped_bar(df, "Departure/Arrival time convenient", "Seat comfort", "satisfaction")

In [ ]:
plot_grouped_bar(df, "Departure/Arrival time convenient", "On-board service", "Class")

In [ ]:
import plotly.express as px

category_columns = ['Type of Travel', 'Class']

# Create a treemap using plotly express
fig = px.treemap(df, path=category_columns)
fig.show()

In [ ]:
value_column = 'Flight Distance'
category_columns = ['Type of Travel', 'Class']

# Create a treemap using plotly express
fig = px.treemap(df, path=category_columns, values=value_column, title=f'Treemap of {value_column} by {", ".join(category_columns)}')
fig.show()

In [ ]:
# List of columns containing scores
score_columns = ['Seat comfort', 'Departure/Arrival time convenient',
       'Food and drink', 'Gate location', 'Inflight wifi service',
       'Inflight entertainment', 'Online support', 'Ease of Online booking',
       'On-board service', 'Leg room service', 'Baggage handling',
       'Checkin service', 'Cleanliness', 'Online boarding']

# Create a new column 'Total_Score' with the sum of scores from specified columns
df['Total_Score'] = df[score_columns].sum(axis=1)

###**Correlation Matrix**

*   Perfect Correlation:

**1.0**: A correlation coefficient of 1.0 indicates a perfect positive linear relationship between two variables. If one variable increases, the other variable also increases proportionally.

**-1.0**: A correlation coefficient of -1.0 indicates a perfect negative linear relationship. If one variable increases, the other variable decreases proportionally.

*   Strong Correlation:

**0.7 to 0.9** (positive or negative): Generally considered a strong correlation. Indicates a significant relationship between the variables.

*   Moderate Correlation:

**0.4 to 0.6**(positive or negative): Moderate correlation. The relationship is noticeable but not as strong.

so we can set criteria for it:

* 1.00 to 0.90 (or -1.00 to -0.90): Very strong positive (or negative) correlation.
* 0.90 to 0.70 (or -0.90 to -0.70): Strong positive (or negative) correlation.
* 0.70 to 0.50 (or -0.70 to -0.50): Moderate positive (or negative) correlation.
* 0.50 to 0.30 (or -0.50 to -0.30): Weak positive (or negative) correlation.

https://doi.org/10.48550/arXiv.2303.01835

In [ ]:
correlation_matrix = df.corr()

# Display the correlation matrix with a color gradient
correlation_matrix.style.background_gradient(cmap='coolwarm').set_precision(2)

In [ ]:
#  'preprocessed_df' is  preprocessed DataFrame
#preprocessed_df.to_csv('preprocessed_data.csv', index=False)


In [ ]:
df



Changing Categorical Data to Numerical

In [ ]:
dfc = df.copy()

#Method one for Label Encoding
dfc.Class.replace('Eco', 0, inplace=True)
dfc.Class.replace('Eco Plus', 1, inplace=True)
dfc.Class.replace('Business', 2, inplace=True)
dfc.Class.astype('int64')

dfc['Customer Type'].replace('Loyal Customer', 1, inplace=True)
dfc['Customer Type'].replace('Disloyal Customer', 0, inplace=True)
dfc['Customer Type'].astype('int64')

#Method two for Label Encoding
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
dfc['satisfaction'] = label_encoder.fit_transform(dfc['satisfaction'])
dfc['Type of Travel'] = label_encoder.fit_transform(dfc['Type of Travel'])

In [ ]:
dfc

In [ ]:
dfc[dfc['satisfaction']==1].count()/dfc[dfc['satisfaction']==0].count()

In [ ]:
df1 = dfc.copy()

### Data visualization 2

In [ ]:
import plotly.express as px
fig = px.scatter(dfc, y='Total_Score', x='Class', color='satisfaction')
fig

In [ ]:
x_column = 'Age'
y_column = 'Flight Distance'
color_column = 'satisfaction'

# Create a hexbin plot with color-coded points
plt.hexbin(dfc[x_column], dfc[y_column], gridsize=(15, 15), cmap='viridis', C=dfc[color_column], alpha=0.8)
plt.xlabel(x_column)
plt.ylabel(y_column)
plt.title(f'Hexbin Plot of {x_column} vs {y_column} (Color-coded by {color_column})')
plt.colorbar(label=color_column)
plt.show()


In [ ]:
value_column = 'satisfaction'
category_columns = ['Age Group', 'Total_Score']

# Create a treemap using plotly express
fig = px.treemap(dfc, path=category_columns, values=value_column, title=f'Treemap of {value_column} by {", ".join(category_columns)}')
fig.show()

In [ ]:
sns.boxplot(x='Age Group', y='Total_Score', data=dfc)
plt.title('Box Plot of Total_Score by Age Group')
plt.xlabel('Age Group')
plt.ylabel('Total_Score')

plt.show()

In [ ]:
sns.boxplot(x='satisfaction', y='Total_Score', data=dfc)
plt.title('Box Plot of satisfaction by Total_Score')
plt.xlabel('satisfaction')
plt.ylabel('Total_Score')

plt.show()

In [ ]:
sns.boxplot(x='satisfaction', y='Flight Distance', data=dfc)
plt.title('Box Plot of satisfaction by Flight Distance')
plt.xlabel('satisfaction')
plt.ylabel('Flight Distance')

plt.show()

In [ ]:
df

Pair plot

In [ ]:
selected_items = ['Arrival Delay in Minutes', 'Departure Delay in Minutes', 'Flight Distance', 'Age']

# Filter the DataFrame for selected items
selected_df = df[selected_items + ['satisfaction']]

# Create a pair plot
pair_plot = sns.pairplot(selected_df, hue='satisfaction', palette={'satisfied': 'green', 'dissatisfied': 'red'})

# Add a legend
pair_plot.add_legend()

# Show the plot
plt.show()

In [ ]:
dfc = dfc.drop('Age Group', axis=1)

In [ ]:
dfc

In [ ]:
df = dfc.copy()
df1 = dfc.copy()
df2 = dfc.copy()
df3 = dfc.copy()


Skew

In [ ]:
import pandas as pd

numeric_cols = dfc.select_dtypes(include=['number']).columns

# Calculate skewness for each numerical column
skewness = dfc[numeric_cols].apply(lambda x: x.skew())

# Display variables with skewness greater than a certain threshold (e.g., 1)
skewed_vars = skewness[abs(skewness) > 1]
print("Skewed Variables:")
print(skewed_vars)


Imbalanced Data and Scaler

In [ ]:
X = dfc.drop(columns = ['satisfaction'])
y = dfc['satisfaction']

#Using SMOTE to balance the Data
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state = 42)
X, y = sm.fit_resample(X, y)
pd.Series(y).value_counts()
dfc = X.join(y)

In [ ]:
#Saving names for rename them after scalling
c = dfc.columns

In [ ]:
from sklearn.preprocessing import MinMaxScaler

Scaler = MinMaxScaler()
dfc = Scaler.fit_transform(dfc)

dfc = pd.DataFrame(dfc)

In [ ]:
dfc.columns = c
dfc

In [ ]:
correlation_matrix = dfc.corr()

# Display the correlation matrix with a color gradient
correlation_matrix.style.background_gradient(cmap='coolwarm').set_precision(2)

plt.figure(figsize=(20, 16))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f', linewidths=.5)
plt.title('Correlation Matrix Heatmap')
plt.show()

# Save the heatmap as an image
plt.savefig('correlation_heatmap.png')

In [ ]:
class_counts = dfc['Customer Type'].value_counts()

# Plot a pie chart
plt.figure(figsize=(8, 8))
plt.pie(class_counts, labels=class_counts.index, autopct='%1.1f%%', startangle=90, colors=sns.color_palette('viridis'))
plt.title('Pie Chart for Customer Type', fontsize=16)
plt.show()

In [ ]:
%matplotlib inline

sns.regplot(x = "Inflight entertainment", y = "satisfaction", data = dfc, ci = None)

In [ ]:
sns.regplot(x = "Class", y = "Type of Travel", data = dfc, ci = None)

In [ ]:
sns.regplot(x = "Seat comfort", y = "Food and drink", data = df, ci = None)

###Feature Engineering

*  Feature importance:

 It is often assessed using machine learning algorithms that provide insights into which features contribute more to the predictive performance. Random Forest is a popular algorithm for this purpose.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Separate features and target variable
X = dfc.drop('satisfaction', axis=1)
y = dfc['satisfaction']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a RandomForestClassifier
rf_model = RandomForestClassifier()

# Fit the model on the training data
rf_model.fit(X_train, y_train)

# Make predictions on the test data
y_pred = rf_model.predict(X_test)

# Evaluate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

# Get feature importances
feature_importances = rf_model.feature_importances_

# Create a DataFrame to display feature importances
feature_importance_df = pd.DataFrame({
    'Feature': X.columns,
    'Importance': feature_importances
})

# Sort the DataFrame by importance in descending order
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

# Display the feature importances
print("Feature Importances:")
print(feature_importance_df)

###Feature selection

Principal Component Analysis (PCA): Linear transformation that projects the data onto a lower-dimensional subspace.

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

# Separate the features from the target variable
X = dfc.drop('satisfaction', axis=1)
y = dfc['satisfaction']

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Apply PCA with the desired number of components
num_components = 20  # Choose the number of components you want
pca = PCA(n_components=num_components)
principal_components = pca.fit_transform(X_scaled)

# Create a DataFrame with the principal components
columns = [f'PC{i+1}' for i in range(num_components)]
pc_df = pd.DataFrame(data=principal_components, columns=columns)

# Concatenate the principal components with the target variable
final_df = pd.concat([pc_df, y], axis=1)

# Visualize the explained variance ratio
explained_variance_ratio = pca.explained_variance_ratio_
print(f"Explained Variance Ratio: {explained_variance_ratio}")

# Plot the cumulative explained variance
cumulative_explained_variance = explained_variance_ratio.cumsum()
plt.plot(range(1, num_components + 1), cumulative_explained_variance, marker='o')
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Explained Variance')
plt.title('Explained Variance vs. Number of Components')
plt.show()

In [ ]:
loadings_pc1 = pca.components_[0]
loadings_df = pd.DataFrame({'Feature': X.columns, 'Loading_PC1': loadings_pc1})
loadings_df

##REGRESSION (satisfaction)

***Train and Test split***

In [ ]:
X = dfc.drop('satisfaction', axis=1)  # Features
y = dfc['satisfaction']  # Target variable

# Split the data into training and testing sets (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Print the shapes of the resulting sets
print("Training set:", X_train.shape, y_train.shape)
print("Testing set:", X_test.shape, y_test.shape)

####Linear Reg
Inflight entertainment & satisfaction

In [ ]:
from sklearn import linear_model

regr = linear_model.LinearRegression()
train_x = np.asanyarray(X_train[['Inflight entertainment']])
train_y = np.asanyarray(y_train)
regr.fit (train_x, train_y)

# The coefficients
print ('Coefficients: ', regr.coef_)
print ('Intercept: ',regr.intercept_)

In linear regression, the coefficients represent the weights assigned to each feature, and the intercept is the value of the predicted response when all independent variables are zero.

In [ ]:
plt.plot(train_x, regr.coef_ * train_x + regr.intercept_, '-r')
plt.xlabel("Inflight entertainment")
plt.ylabel("satisfaction")

####Evaluation Linear

In [ ]:
from sklearn.metrics import r2_score

test_x = np.asanyarray(X_test[['Inflight entertainment']])
test_y = np.asanyarray(y_test)
test_y_ = regr.predict(test_x)

print("Mean absolute error: %.2f" % np.mean(np.absolute(test_y_ - test_y)))
print("Residual sum of squares (MSE): %.2f" % np.mean((test_y_ - test_y) ** 2))
print("R2-score: %.2f" % r2_score(test_y , test_y_) )



* **Interpretation:**

 The MAE and MSE provide insights into the magnitude of errors, with lower values indicating better performance.
The R2-score gives an idea of the proportion of variability explained by  model. An R2-score of 0.27 suggests that there may be room for improvement.

Seat comfort & Food and drink

In [ ]:
regr = linear_model.LinearRegression()
train_x = np.asanyarray(X_train[['Seat comfort']])
train_y = np.asanyarray(X_train[['Food and drink']])
regr.fit (train_x, train_y)

# The coefficients
print ('Coefficients: ', regr.coef_)
print ('Intercept: ',regr.intercept_)

In [ ]:
plt.plot(train_x, regr.coef_ * train_x + regr.intercept_, '-r')
plt.xlabel("Seat comfort")
plt.ylabel("Food and drink")

In [ ]:
test_x = np.asanyarray(X_test[['Seat comfort']])
test_y = np.asanyarray(X_test[['Food and drink']])
test_y_ = regr.predict(test_x)

print("Mean absolute error: %.2f" % np.mean(np.absolute(test_y_ - test_y)))
print("Residual sum of squares (MSE): %.2f" % np.mean((test_y_ - test_y) ** 2))
print("R2-score: %.2f" % r2_score(test_y , test_y_) )

Online boarding & Inflight wifi service

In [ ]:
regr = linear_model.LinearRegression()
train_x = np.asanyarray(X_train[['Online boarding']])
train_y = np.asanyarray(X_train[['Inflight wifi service']])
regr.fit (train_x, train_y)

# The coefficients
print ('Coefficients: ', regr.coef_)
print ('Intercept: ',regr.intercept_)

In [ ]:
plt.plot(train_x, regr.coef_ * train_x + regr.intercept_, '-r')
plt.xlabel("Online boarding")
plt.ylabel("Inflight wifi service")

In [ ]:
test_x = np.asanyarray(X_test[['Online boarding']])
test_y = np.asanyarray(X_test[['Inflight wifi service']])
test_y_ = regr.predict(test_x)

print("Mean absolute error: %.2f" % np.mean(np.absolute(test_y_ - test_y)))
print("Residual sum of squares (MSE): %.2f" % np.mean((test_y_ - test_y) ** 2))
print("R2-score: %.2f" % r2_score(test_y , test_y_) )

##REGRESSION (Total_Score)



 **5 common regression models:**
 1. Linear Regression
 2. Decision Tree Regressor
 3. Random Forest Regressor
 4. Support Vector Regressor (SVR) --  high run time!!! --
 5. Gradient Boosting Regressor

Bayesian Ridge

In [ ]:
from sklearn.linear_model import BayesianRidge
from sklearn.model_selection import KFold, GridSearchCV, cross_val_predict
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import numpy as np
import matplotlib.pyplot as plt

#  dfc is  DataFrame with features and target variable
X = df.drop('Total_Score', axis=1)
y = df['Total_Score']

# Initialize KFold cross-validation
k_folds = 10
kf = KFold(n_splits=k_folds, shuffle=True, random_state=42)

# Initialize lists to store evaluation metrics for each fold
mape_scores = []
mae_scores = []
r2_scores = []
mse_scores = []
mspe_scores = []

# Define the hyperparameter grid
param_grid = {
    'alpha_1': [1e-6, 1e-4, 1e-2, 1e-1, 1, 10, 100],
    'alpha_2': [1e-6, 1e-4, 1e-2, 1e-1, 1, 10, 100],
    'lambda_1': [1e-6, 1e-4, 1e-2, 1e-1, 1, 10, 100],
    'lambda_2': [1e-6, 1e-4, 1e-2, 1e-1, 1, 10, 100]
}

# Perform hyperparameter tuning with GridSearchCV
grid_search = GridSearchCV(estimator=BayesianRidge(), param_grid=param_grid, scoring='neg_mean_squared_error', cv=k_folds)
grid_search.fit(X, y)

# Get the best hyperparameters
best_hyperparameters = grid_search.best_params_

# Perform k-fold cross-validation
for train_index, val_index in kf.split(X):
    X_train_fold, X_val_fold = X.iloc[train_index], X.iloc[val_index]
    y_train_fold, y_val_fold = y.iloc[train_index], y.iloc[val_index]

    # Use the best hyperparameters to train the final model on the training set
    bayesian_ridge_model = BayesianRidge(**best_hyperparameters)
    bayesian_ridge_model.fit(X_train_fold, y_train_fold)

    # Make predictions on the validation set
    y_val_pred = bayesian_ridge_model.predict(X_val_fold)

    # Calculate evaluation metrics for the fold
    fold_mape = np.mean(np.abs((y_val_fold - y_val_pred) / y_val_fold)) * 100
    fold_mae = mean_absolute_error(y_val_fold, y_val_pred)
    fold_r2 = r2_score(y_val_fold, y_val_pred)
    fold_mse = mean_squared_error(y_val_fold, y_val_pred)

    # Calculate MSPE
    fold_mspe = np.mean(((y_val_fold - y_val_pred) / y_val_fold) ** 2) * 100

    mape_scores.append(fold_mape)
    mae_scores.append(fold_mae)
    r2_scores.append(fold_r2)
    mse_scores.append(fold_mse)
    mspe_scores.append(fold_mspe)

# Calculate mean and standard deviation of evaluation metrics
mean_mape = np.mean(mape_scores)
mean_mae = np.mean(mae_scores)
mean_r2 = np.mean(r2_scores)
mean_mse = np.mean(mse_scores)
std_mse = np.std(mse_scores)
mean_mspe = np.mean(mspe_scores)

# Print the best hyperparameters
print(f"Best Hyperparameters: {best_hyperparameters}")

# Print the evaluation metrics
print(f"Mean Squared Error: {mean_mse:.4f}")
print(f"Standard Deviation of Mean Squared Error: {std_mse:.4f}")
print(f"R-squared (R2) Score: {mean_r2:.4f}")
print(f"Mean Absolute Error: {mean_mae:.4f}")
print(f"Mean Absolute Percentage Error (MAPE): {mean_mape:.4f}%")
print(f"Mean Squared Percentage Error (MSPE): {mean_mspe:.4f}")
print("\n")

# Optional: Plot predicted vs. actual values for better visualization
y_pred = cross_val_predict(bayesian_ridge_model, X, y, cv=k_folds)
plt.scatter(y, y_pred)
plt.xlabel("Actual Values")
plt.ylabel("Predicted Values")
plt.title("Bayesian Ridge Regression - Predicted vs. Actual Values")
plt.show()


Elastic Net

In [ ]:
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import GridSearchCV, KFold, train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import numpy as np
import matplotlib.pyplot as plt


X = df.drop('Total_Score', axis=1)
y = df['Total_Score']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the Elastic Net regression model
elastic_net_model = ElasticNet()

# Define the hyperparameter grid
param_grid = {
    'alpha': [0.1, 1, 10, 100],
    'l1_ratio': [0.1, 0.5, 0.7, 0.9]
}

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=elastic_net_model, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5)

# Fit the model to the training data
grid_search.fit(X_train, y_train)

# Get the best hyperparameters
best_alpha = grid_search.best_params_['alpha']
best_l1_ratio = grid_search.best_params_['l1_ratio']

# Initialize KFold cross-validation
k_folds = 10
kf = KFold(n_splits=k_folds, shuffle=True, random_state=42)

# Initialize lists to store evaluation metrics for each fold
mape_scores = []
mae_scores = []
r2_scores = []
mse_scores = []
mspe_scores = []

# Perform k-fold cross-validation
for train_index, val_index in kf.split(X_train):
    X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]
    y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]

    # Use the best hyperparameters to train the final model on the validation set
    final_model = ElasticNet(alpha=best_alpha, l1_ratio=best_l1_ratio)
    final_model.fit(X_train_fold, y_train_fold)

    # Make predictions on the validation set
    y_val_pred = final_model.predict(X_val_fold)

    # Calculate evaluation metrics for the fold
    fold_mape = np.mean(np.abs((y_val_fold - y_val_pred) / y_val_fold)) * 100
    fold_mae = mean_absolute_error(y_val_fold, y_val_pred)
    fold_r2 = r2_score(y_val_fold, y_val_pred)
    fold_mse = mean_squared_error(y_val_fold, y_val_pred)

    # Calculate MSPE
    fold_mspe = np.mean(((y_val_fold - y_val_pred) / y_val_fold) ** 2) * 100

    mape_scores.append(fold_mape)
    mae_scores.append(fold_mae)
    r2_scores.append(fold_r2)
    mse_scores.append(fold_mse)
    mspe_scores.append(fold_mspe)

# Calculate mean and standard deviation of evaluation metrics
mean_mape = np.mean(mape_scores)
mean_mae = np.mean(mae_scores)
mean_r2 = np.mean(r2_scores)
mean_mse = np.mean(mse_scores)
std_mse = np.std(mse_scores)
mean_mspe = np.mean(mspe_scores)

# Print the best hyperparameters and evaluation metrics
print(f"Best alpha: {best_alpha}")
print(f"Best l1_ratio: {best_l1_ratio}")
print(f"Mean Squared Error: {mean_mse:.4f}")
print(f"Standard Deviation of Mean Squared Error: {std_mse:.4f}")
print(f"R-squared (R2) Score: {mean_r2:.4f}")
print(f"Mean Absolute Error: {mean_mae:.4f}")
print(f"Mean Absolute Percentage Error (MAPE): {mean_mape:.4f}%")
print(f"Mean Squared Percentage Error (MSPE): {mean_mspe:.4f}")
print("\n")

# Use the best hyperparameters to train the final model on the entire training set
final_model = ElasticNet(alpha=best_alpha, l1_ratio=best_l1_ratio)
final_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = final_model.predict(X_test)



In [ ]:
# Optional: Plot predicted vs. actual values for better visualization
y_pred = final_model.predict(X_test)
plt.scatter(y_test, y_pred)
plt.xlabel("Actual Values")
plt.ylabel("Predicted Values")
plt.title("Elastic Regressor - Predicted vs. Actual Values")
plt.show()

KNeighbors Regressor

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV, KFold, train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error



X = df.drop('Total_Score', axis=1)
y = df['Total_Score']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the KNN Regressor model
knn_model = KNeighborsRegressor()

# Define the hyperparameter grid
param_grid = {
    'n_neighbors': [3, 5, 7, 9],
    'weights': ['uniform', 'distance'],
    'p': [1, 2]
}

# Initialize KFold cross-validation
k_folds = 10
kf = KFold(n_splits=k_folds, shuffle=True, random_state=42)

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=knn_model, param_grid=param_grid, scoring='neg_mean_squared_error', cv=k_folds)

# Initialize lists to store evaluation metrics for each fold
mae_scores = []
r2_scores = []
mse_scores = []
mape_scores = []
mspe_scores = []

# Perform k-fold cross-validation
for train_index, val_index in kf.split(X_train):
    X_train_fold, X_val_fold = X.iloc[train_index], X.iloc[val_index]
    y_train_fold, y_val_fold = y.iloc[train_index], y.iloc[val_index]

    # Fit the model to the training data
    grid_search.fit(X_train_fold, y_train_fold)

    # Make predictions on the validation set
    y_val_pred = grid_search.predict(X_val_fold)

    # Calculate evaluation metrics for the fold
    fold_mae = mean_absolute_error(y_val_fold, y_val_pred)
    fold_r2 = r2_score(y_val_fold, y_val_pred)
    fold_mse = mean_squared_error(y_val_fold, y_val_pred)

    # Calculate MSPE
    fold_mspe = np.mean(((y_val_fold - y_val_pred) / y_val_fold) ** 2) * 100

    # Calculate MAPE
    fold_mape = np.mean(np.abs((y_val_fold - y_val_pred) / y_val_fold)) * 100

    mae_scores.append(fold_mae)
    r2_scores.append(fold_r2)
    mse_scores.append(fold_mse)
    mspe_scores.append(fold_mspe)
    mape_scores.append(fold_mape)

# Calculate mean and standard deviation of evaluation metrics
mean_mae = np.mean(mae_scores)
mean_r2 = np.mean(r2_scores)
mean_mse = np.mean(mse_scores)
std_mse = np.std(mse_scores)
mean_mspe = np.mean(mspe_scores)
mean_mape = np.mean(mape_scores)

# Print the evaluation metrics
print(f"Mean Squared Error: {mean_mse:.4f}")
print(f"Standard Deviation of Mean Squared Error: {std_mse:.4f}")
print(f"R-squared (R2) Score: {mean_r2:.4f}")
print(f"Mean Absolute Error: {mean_mae:.4f}")
print(f"Mean Absolute Percentage Error (MAPE): {mean_mape:.4f}%")
print(f"Mean Squared Percentage Error (MSPE): {mean_mspe:.4f}")
print("\n")


In [ ]:
# Plot predicted vs. actual values for better visualization
y_pred = grid_search.predict(X_test)

plt.scatter(y_test, y_pred)
plt.xlabel("Actual Values")
plt.ylabel("Predicted Values")
plt.title("KNeighbors Regressor - Predicted vs. Actual Values")
plt.show()


Decision Tree Regressor

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_absolute_percentage_error
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt


X = df.drop('Total_Score', axis=1)
y = df['Total_Score']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the Decision Tree Regressor model
dt_regressor = DecisionTreeRegressor()

# Define the hyperparameter grid
param_grid = {
    'max_depth': [None, 5, 10, 15],
    'min_samples_split': [2, 5, 10]
}

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=dt_regressor, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5)

# Fit the model to the training data
grid_search.fit(X_train, y_train)

# Get the best hyperparameters
best_params = grid_search.best_params_
best_max_depth = best_params['max_depth']
best_min_samples_split = best_params['min_samples_split']

# Initialize KFold cross-validation
k_folds = 10
kf = KFold(n_splits=k_folds, shuffle=True, random_state=42)

# Initialize an array to store metrics for each fold
mse_scores = []
r2_scores = []
mae_scores = []
mape_scores = []
mspe_scores = []

# Perform k-fold cross-validation
for train_index, val_index in kf.split(X):
    X_train_fold, X_val_fold = X.iloc[train_index], X.iloc[val_index]
    y_train_fold, y_val_fold = y.iloc[train_index], y.iloc[val_index]

    # Initialize Decision Tree Regressor with the best hyperparameters
    final_model = DecisionTreeRegressor(max_depth=best_max_depth, min_samples_split=best_min_samples_split)

    # Fit the model to the training fold
    final_model.fit(X_train_fold, y_train_fold)

    # Make predictions on the validation fold
    y_val_pred = final_model.predict(X_val_fold)

    # Calculate evaluation metrics for the fold
    fold_mse = mean_squared_error(y_val_fold, y_val_pred)
    fold_r2 = r2_score(y_val_fold, y_val_pred)
    fold_mae = mean_absolute_error(y_val_fold, y_val_pred)
    fold_mape = mean_absolute_percentage_error(y_val_fold, y_val_pred)
    fold_mspe = (fold_mape / 100) ** 2  # Calculating MSPE from MAPE

    mse_scores.append(fold_mse)
    r2_scores.append(fold_r2)
    mae_scores.append(fold_mae)
    mape_scores.append(fold_mape)
    mspe_scores.append(fold_mspe)

# Calculate mean and standard deviation of metrics
mean_mse = pd.Series(mse_scores).mean()
std_mse = pd.Series(mse_scores).std()
mean_r2 = pd.Series(r2_scores).mean()
mean_mae = pd.Series(mae_scores).mean()
mean_mape = pd.Series(mape_scores).mean()
mean_mspe = pd.Series(mspe_scores).mean()

# Print the best hyperparameters and evaluation metrics
print(f"Best Max Depth: {best_max_depth}")
print(f"Best Min Samples Split: {best_min_samples_split}")
print(f"Mean Squared Error: {mean_mse:.4f}")
print(f"Standard Deviation of Mean Squared Error: {std_mse:.4f}")
print(f"R-squared (R2) Score: {mean_r2:.4f}")
print(f"Mean Absolute Error: {mean_mae:.4f}")
print(f"Mean Absolute Percentage Error (MAPE): {mean_mape:.4f}%")
print(f"Mean Squared Percentage Error (MSPE): {mean_mspe:.4f}")
print("\n")

# Optional: Plot predicted vs. actual values for better visualization
y_pred = grid_search.predict(X_test)
plt.scatter(y_test, y_pred)
plt.xlabel("Actual Values")
plt.ylabel("Predicted Values")
plt.title("Decision Tree Regressor - Predicted vs. Actual Values")
plt.show()


In [ ]:
plt.scatter(y_test, y_pred)
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], linestyle='--', color='red', linewidth=2)  # Line of unity
plt.xlabel("Actual Values")
plt.ylabel("Predicted Values")
plt.title("Decision Tree Regressor - Predicted vs. Actual Values")
plt.show()

Random Forest Regressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV, train_test_split, KFold
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import numpy as np
import matplotlib.pyplot as plt

X = df.drop('Total_Score', axis=1)
y = df['Total_Score']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the Random Forest Regressor model
rf_regressor = RandomForestRegressor()

# Define the hyperparameter grid
param_dist = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Initialize RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=rf_regressor, param_distributions=param_dist, n_iter=10, scoring='neg_mean_squared_error', cv=5, n_jobs=-1, random_state=42)

# Fit the model to the training data
random_search.fit(X_train, y_train)

# Get the best hyperparameters
best_params = random_search.best_params_

# Initialize KFold cross-validation
k_folds = 10
kf = KFold(n_splits=k_folds, shuffle=True, random_state=42)

# Initialize an array to store metrics for each fold
mse_scores = []
r2_scores = []
mae_scores = []
mape_scores = []
mspe_scores = []

# Perform k-fold cross-validation
for train_index, val_index in kf.split(X):
    X_train_fold, X_val_fold = X.iloc[train_index], X.iloc[val_index]
    y_train_fold, y_val_fold = y.iloc[train_index], y.iloc[val_index]

    # Use the best hyperparameters to train the final model
    final_model = RandomForestRegressor(**best_params)
    final_model.fit(X_train_fold, y_train_fold)

    # Make predictions on the validation set
    y_val_pred = final_model.predict(X_val_fold)

    # Calculate evaluation metrics for the fold
    fold_mse = mean_squared_error(y_val_fold, y_val_pred)
    fold_r2 = r2_score(y_val_fold, y_val_pred)
    fold_mae = mean_absolute_error(y_val_fold, y_val_pred)

    # Calculate MAPE
    fold_mape = np.mean(np.abs((y_val_fold - y_val_pred) / y_val_fold)) * 100

    # Calculate MSPE
    fold_mspe = np.mean(((y_val_fold - y_val_pred) / y_val_fold) ** 2) * 100

    mse_scores.append(fold_mse)
    r2_scores.append(fold_r2)
    mae_scores.append(fold_mae)
    mape_scores.append(fold_mape)
    mspe_scores.append(fold_mspe)

# Calculate mean and standard deviation of metrics
mean_mse = np.mean(mse_scores)
std_mse = np.std(mse_scores)

mean_r2 = np.mean(r2_scores)
std_r2 = np.std(r2_scores)

mean_mae = np.mean(mae_scores)
std_mae = np.std(mae_scores)

mean_mape = np.mean(mape_scores)
mean_mspe = np.mean(mspe_scores)

# Print the best hyperparameters and evaluation metrics
print(f"Best Hyperparameters: {best_params}")
print(f"Mean Squared Error: {mean_mse:.4f} (±{std_mse:.4f})")
print(f"R-squared: {mean_r2:.4f} (±{std_r2:.4f})")
print(f"Mean Absolute Error: {mean_mae:.4f} (±{std_mae:.4f})")
print(f"Mean Absolute Percentage Error (MAPE): {mean_mape:.4f}%")
print(f"Mean Squared Percentage Error (MSPE): {mean_mspe:.4f}%")
print("\n")

# Optional: Plot predicted vs. actual values for better visualization
y_pred = final_model.predict(X_test)
plt.scatter(y_test, y_pred)
plt.xlabel("Actual Values")
plt.ylabel("Predicted Values")
plt.title("Random Forest Regressor - Predicted vs. Actual Values")
plt.show()


SVR

In [ ]:
from sklearn.svm import SVR
from sklearn.model_selection import RandomizedSearchCV, KFold
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import train_test_split


X = df.drop('Total_Score', axis=1)
y = df['Total_Score']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the Support Vector Regression model
svr_model = SVR()

# Define the hyperparameter grid
param_dist = {
    'C': np.logspace(-3, 3, 7),
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'gamma': ['scale', 'auto'] + list(np.logspace(-3, 3, 7))
}

# Initialize RandomizedSearchCV
random_search = RandomizedSearchCV(
    svr_model,
    param_distributions=param_dist,
    n_iter=20,
    cv=5,
    scoring='neg_mean_squared_error',
    verbose=1,
    n_jobs=-1,
    random_state=42
)

# Fit the model with hyperparameter tuning on the training set
random_search.fit(X_train, y_train)

# Get the best hyperparameters
best_params = random_search.best_params_

# Initialize KFold cross-validation
k_folds = 10
kf = KFold(n_splits=k_folds, shuffle=True, random_state=42)

# Initialize an array to store mean squared error for each fold
mse_scores = []

# Perform k-fold cross-validation
for train_index, val_index in kf.split(X):
    X_train_fold, X_val_fold = X.iloc[train_index], X.iloc[val_index]
    y_train_fold, y_val_fold = y.iloc[train_index], y.iloc[val_index]

    # Use the best hyperparameters to train the final model
    final_model = SVR(**best_params)
    final_model.fit(X_train_fold, y_train_fold)

    # Make predictions on the validation set
    y_val_pred = final_model.predict(X_val_fold)

    # Calculate mean squared error for the fold
    fold_mse = mean_squared_error(y_val_fold, y_val_pred)
    mse_scores.append(fold_mse)

# Calculate mean and standard deviation of mean squared error scores
mean_mse = np.mean(mse_scores)
std_mse = np.std(mse_scores)

# Print the best hyperparameters and evaluation metrics
print(f'Best Hyperparameters: {best_params}')
print(f'Mean Squared Error: {mean_mse:.4f}')
print(f'Standard Deviation of Mean Squared Error: {std_mse:.4f}')

# Use the best hyperparameters to train the final model on the entire train
final_model = SVR(**best_params)
final_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = final_model.predict(X_test)

# Evaluate the model on the test set
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print(f'Mean Squared Error on Test Set: {mse:.4f}')
print(f'R-squared Score on Test Set: {r2:.4f}')
print(f'Mean Absolute Error on Test Set: {mae:.4f}')

In [ ]:
# Optional: Plot predicted vs. actual values for better visualization
y_pred = final_model.predict(X_test)
plt.scatter(y_test, y_pred)
plt.xlabel("Actual Values")
plt.ylabel("Predicted Values")
plt.title(" Gradient Boosting Regressor - Predicted vs. Actual Values")
plt.show()

2 Sampling Methods


In [ ]:
import pandas as pd

def simple_random_sampling(data, sample_size):
    # Randomly sample from the entire dataset
    random_sample = data.sample(sample_size, random_state=42)  # Setting a random state for reproducibility
    return random_sample

def stratified_random_sampling(data, strata_col, sample_size):
    # Create an empty DataFrame to store the sampled data
    sampled_data = pd.DataFrame()

    # Identify unique strata in the dataset
    strata_values = data[strata_col].unique()

    # Perform stratified random sampling for each stratum
    for stratum in strata_values:
        # Select data points belonging to the current stratum
        stratum_data = data[data[strata_col] == stratum]

        # Determine the number of samples to draw from the stratum
        stratum_sample_size = int(sample_size * len(stratum_data) / len(data))

        # Randomly sample from the current stratum
        stratum_sample = stratum_data.sample(stratum_sample_size, random_state=42)  # Setting a random state for reproducibility

        # Append the stratum sample to the overall sampled data
        sampled_data = pd.concat([sampled_data, stratum_sample])

    return sampled_data


Gradient Boosting Regressor

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import RandomizedSearchCV, train_test_split, KFold
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import numpy as np

X = df.drop('Total_Score', axis=1)
y = df['Total_Score']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the Gradient Boosting Regressor model
gb_model = GradientBoostingRegressor()

# Define the hyperparameter grid
param_dist = {
    'n_estimators': [50, 100, 150, 200],
    'learning_rate': [0.01, 0.1, 0.2, 0.3],
    'max_depth': [3, 4, 5, 6],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'subsample': [0.8, 0.9, 1.0]
}

# Initialize RandomizedSearchCV
random_search = RandomizedSearchCV(gb_model, param_distributions=param_dist, n_iter=20, cv=5, scoring='neg_mean_squared_error', verbose=1, n_jobs=-1, random_state=42)

# Fit the model with hyperparameter tuning on the training set
random_search.fit(X_train, y_train)

# Get the best hyperparameters
best_params = random_search.best_params_

# Initialize KFold cross-validation
k_folds = 10
kf = KFold(n_splits=k_folds, shuffle=True, random_state=42)

# Initialize an array to store mean squared error for each fold
mse_scores = []

# Perform k-fold cross-validation
for train_index, val_index in kf.split(X):
    X_train_fold, X_val_fold = X.iloc[train_index], X.iloc[val_index]
    y_train_fold, y_val_fold = y.iloc[train_index], y.iloc[val_index]

    # Use the best hyperparameters to train the final model
    final_model = GradientBoostingRegressor(**best_params)
    final_model.fit(X_train_fold, y_train_fold)

    # Make predictions on the validation set
    y_val_pred = final_model.predict(X_val_fold)

    # Calculate mean squared error for the fold
    fold_mse = mean_squared_error(y_val_fold, y_val_pred)
    mse_scores.append(fold_mse)

# Calculate mean and standard deviation of mean squared error scores
mean_mse = np.mean(mse_scores)
std_mse = np.std(mse_scores)

print(f'Best Hyperparameters: {best_params}')
print(f'Mean Squared Error: {mean_mse:.4f}')
print(f'Standard Deviation of Mean Squared Error: {std_mse:.4f}')

# Use the best hyperparameters to train the final model on the entire training set
final_model = GradientBoostingRegressor(**best_params)
final_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = final_model.predict(X_test)

# Evaluate the model on the test set
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print(f'Mean Squared Error on Test Set: {mse:.4f}')
print(f'R-squared Score on Test Set: {r2:.4f}')
print(f'Mean Absolute Error on Test Set: {mae:.4f}')


In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import RandomizedSearchCV, train_test_split, KFold
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import numpy as np

X = df.drop('Total_Score', axis=1)
y = df['Total_Score']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the Gradient Boosting Regressor model
gb_model = GradientBoostingRegressor()

# Define the hyperparameter grid
param_dist = {
    'n_estimators': [50, 100, 150, 200],
    'learning_rate': [0.01, 0.1, 0.2, 0.3],
    'max_depth': [3, 4, 5, 6],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'subsample': [0.8, 0.9, 1.0]
}

# Initialize RandomizedSearchCV
random_search = RandomizedSearchCV(gb_model, param_distributions=param_dist, n_iter=20, cv=5, scoring='neg_mean_squared_error', verbose=1, n_jobs=-1, random_state=42)

# Fit the model with hyperparameter tuning on the training set
random_search.fit(X_train, y_train)

# Get the best hyperparameters
best_params = random_search.best_params_

# Initialize KFold cross-validation
k_folds = 10
kf = KFold(n_splits=k_folds, shuffle=True, random_state=42)

# Initialize arrays to store evaluation metrics for each fold
mse_scores = []
r2_scores = []
mae_scores = []
mape_scores = []
mspe_scores = []

# Perform k-fold cross-validation
for train_index, val_index in kf.split(X):
    X_train_fold, X_val_fold = X.iloc[train_index], X.iloc[val_index]
    y_train_fold, y_val_fold = y.iloc[train_index], y.iloc[val_index]

    # Use the best hyperparameters to train the final model
    final_model = GradientBoostingRegressor(**best_params)
    final_model.fit(X_train_fold, y_train_fold)

    # Make predictions on the validation set
    y_val_pred = final_model.predict(X_val_fold)

    # Calculate evaluation metrics for the fold
    fold_mse = mean_squared_error(y_val_fold, y_val_pred)
    fold_r2 = r2_score(y_val_fold, y_val_pred)
    fold_mae = mean_absolute_error(y_val_fold, y_val_pred)

    # Calculate MAPE
    fold_mape = np.mean(np.abs((y_val_fold - y_val_pred) / y_val_fold)) * 100

    # Calculate MSPE
    fold_mspe = np.mean(((y_val_fold - y_val_pred) / y_val_fold) ** 2) * 100

    mse_scores.append(fold_mse)
    r2_scores.append(fold_r2)
    mae_scores.append(fold_mae)
    mape_scores.append(fold_mape)
    mspe_scores.append(fold_mspe)

# Calculate mean and standard deviation of metrics
mean_mse = np.mean(mse_scores)
std_mse = np.std(mse_scores)

mean_r2 = np.mean(r2_scores)
std_r2 = np.std(r2_scores)

mean_mae = np.mean(mae_scores)
std_mae = np.std(mae_scores)

mean_mape = np.mean(mape_scores)
mean_mspe = np.mean(mspe_scores)

# Print the best hyperparameters and evaluation metrics
print(f'Best Hyperparameters: {best_params}')
print(f'Mean Squared Error: {mean_mse:.4f} (±{std_mse:.4f})')
print(f'R-squared: {mean_r2:.4f} (±{std_r2:.4f})')
print(f'Mean Absolute Error: {mean_mae:.4f} (±{std_mae:.4f})')
print(f'Mean Absolute Percentage Error (MAPE): {mean_mape:.4f}%')
print(f'Mean Squared Percentage Error (MSPE): {mean_mspe:.4f}%')
print("\n")

# Use the best hyperparameters to train the final model on the entire training set
final_model = GradientBoostingRegressor(**best_params)
final_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = final_model.predict(X_test)

# Evaluate the model on the test set
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print(f'Mean Squared Error on Test Set: {mse:.4f}')
print(f'R-squared Score on Test Set: {r2:.4f}')
print(f'Mean Absolute Error on Test Set: {mae:.4f}')


In [ ]:
# Optional: Plot predicted vs. actual values for better visualization
y_pred = final_model.predict(X_test)
plt.scatter(y_test, y_pred)
plt.xlabel("Actual Values")
plt.ylabel("Predicted Values")
plt.title(" Gradient Boosting Regressor - Predicted vs. Actual Values")
plt.show()

poission reg

In [ ]:
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.linear_model import PoissonRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

X = df1.drop('Total_Score', axis=1)
y = df1['Total_Score']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the hyperparameter grid for grid search
param_grid = {
    'alpha': [0.01, 0.1, 1.0, 10.0],
    'tol': [1e-4, 1e-3, 1e-2]
}

# Initialize Poisson Regressor
poisson_regressor = PoissonRegressor()

# Initialize GridSearchCV
grid_search = GridSearchCV(poisson_regressor, param_grid, scoring='neg_mean_squared_error', cv=5, n_jobs=-1)

# Fit the model with hyperparameter tuning on the training set
grid_search.fit(X_train, y_train)

# Get the best hyperparameters
best_params = grid_search.best_params_

# Use k-fold cross-validation for final evaluation
k_folds = 10
kf = KFold(n_splits=k_folds, shuffle=True, random_state=42)

# Initialize lists to store evaluation metrics for each fold
mae_scores = []
mse_scores = []
r2_scores = []
mape_scores = []
mspe_scores = []

# Perform k-fold cross-validation
for fold, (train_index, val_index) in enumerate(kf.split(X_train)):
    X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]
    y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]

    # Use the best hyperparameters to train the final model on the validation set
    final_model = PoissonRegressor(**best_params)
    final_model.fit(X_train_fold, y_train_fold)

    # Make predictions on the validation set
    y_val_pred = final_model.predict(X_val_fold)

    # Calculate evaluation metrics for the fold
    fold_mae = mean_absolute_error(y_val_fold, y_val_pred)
    fold_mse = mean_squared_error(y_val_fold, y_val_pred)
    fold_r2 = r2_score(y_val_fold, y_val_pred)

    # Calculate MAPE
    fold_mape = np.mean(np.abs((y_val_fold - y_val_pred) / y_val_fold)) * 100

    # Calculate MSPE
    fold_mspe = np.mean(((y_val_fold - y_val_pred) / y_val_fold) ** 2) * 100

    mae_scores.append(fold_mae)
    mse_scores.append(fold_mse)
    r2_scores.append(fold_r2)
    mape_scores.append(fold_mape)
    mspe_scores.append(fold_mspe)

    print(f'Mean Absolute Error for Fold {fold + 1}: {fold_mae}')
    print(f'Mean Squared Error for Fold {fold + 1}: {fold_mse}')
    print(f'R-squared for Fold {fold + 1}: {fold_r2}')
    print(f'MAPE for Fold {fold + 1}: {fold_mape}%')
    print(f'MSPE for Fold {fold + 1}: {fold_mspe}%')

# Calculate mean and standard deviation of evaluation metrics
mean_mae = np.mean(mae_scores)
std_mae = np.std(mae_scores)
mean_mse = np.mean(mse_scores)
std_mse = np.std(mse_scores)
mean_r2 = np.mean(r2_scores)
std_r2 = np.std(r2_scores)
mean_mape = np.mean(mape_scores)
mean_mspe = np.mean(mspe_scores)

print(f'Mean Mean Absolute Error: {mean_mae}')
print(f'Standard Deviation of Mean Absolute Error: {std_mae}')
print(f'Mean Mean Squared Error: {mean_mse}')
print(f'Standard Deviation of Mean Squared Error: {std_mse}')
print(f'Mean R-squared: {mean_r2}')
print(f'Standard Deviation of R-squared: {std_r2}')
print(f'Mean MAPE: {mean_mape}%')
print(f'Mean MSPE: {mean_mspe}%')

# Use the best hyperparameters to train the final model on the entire training set
final_model = PoissonRegressor(**best_params)
final_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = final_model.predict(X_test)

# Evaluate the model on the test set
test_mae = mean_absolute_error(y_test, y_pred)
test_mse = mean_squared_error(y_test, y_pred)
test_r2 = r2_score(y_test, y_pred)

# Calculate MAPE for the test set
test_mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100

# Calculate MSPE for the test set
test_mspe = np.mean(((y_test - y_pred) / y_test) ** 2) * 100

print(f'Best Hyperparameters: {best_params}')
print(f'Mean Absolute Error on Test Set: {test_mae}')
print(f'Mean Squared Error on Test Set: {test_mse}')
print(f'R-squared on Test Set: {test_r2}')
print(f'MAPE on Test Set: {test_mape}%')
print(f'MSPE on Test Set: {test_mspe}%')


In [ ]:
# Optional: Plot predicted vs. actual values for better visualization
y_pred = final_model.predict(X_test)
plt.scatter(y_test, y_pred)
plt.xlabel("Actual Values")
plt.ylabel("Predicted Values")
plt.title(" poission Regressor - Predicted vs. Actual Values")
plt.show()

MLPREg

In [ ]:
from sklearn.model_selection import KFold, RandomizedSearchCV
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


X = df.drop('Total_Score', axis=1)
y = df['Total_Score']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# --- MLP Regressor ---

# Define the hyperparameter grid for randomized search
param_dist_mlp = {
    'hidden_layer_sizes': [(50,), (100,), (50, 50), (100, 50)],
    'activation': ['logistic', 'tanh', 'relu'],
    'alpha': np.logspace(-4, 0, 5),
}

# Initialize MLP Regressor
mlp_regressor = MLPRegressor(max_iter=1000, random_state=42)

# Initialize RandomizedSearchCV
random_search_mlp = RandomizedSearchCV(mlp_regressor, param_distributions=param_dist_mlp, n_iter=10, cv=5, scoring='neg_mean_squared_error', random_state=42, n_jobs=-1)

# Fit the model with hyperparameter tuning on the training set
random_search_mlp.fit(X_train, y_train)

# Get the best hyperparameters for MLP Regressor
best_params_mlp = random_search_mlp.best_params_

# Initialize KFold cross-validation for MLP Regressor
k_folds_mlp = 10
kf_mlp = KFold(n_splits=k_folds_mlp, shuffle=True, random_state=42)

# Initialize lists to store regression metrics for each fold
mse_scores_mlp = []
mae_scores_mlp = []
r2_scores_mlp = []

# Perform k-fold cross-validation for MLP Regressor
for fold, (train_index, val_index) in enumerate(kf_mlp.split(X_train)):
    X_train_fold_mlp, X_val_fold_mlp = X_train.iloc[train_index], X_train.iloc[val_index]
    y_train_fold_mlp, y_val_fold_mlp = y_train.iloc[train_index], y_train.iloc[val_index]

    # Initialize MLP Regressor with the best hyperparameters
    final_model_mlp = MLPRegressor(max_iter=1000, random_state=42, **best_params_mlp)

    # Fit the model to the training fold
    final_model_mlp.fit(X_train_fold_mlp, y_train_fold_mlp)

    # Make predictions on the validation fold
    y_val_pred_mlp = final_model_mlp.predict(X_val_fold_mlp)

    # Calculate regression metrics for the fold
    fold_mse_mlp = mean_squared_error(y_val_fold_mlp, y_val_pred_mlp)
    fold_mae_mlp = mean_absolute_error(y_val_fold_mlp, y_val_pred_mlp)
    fold_r2_mlp = r2_score(y_val_fold_mlp, y_val_pred_mlp)

    mse_scores_mlp.append(fold_mse_mlp)
    mae_scores_mlp.append(fold_mae_mlp)
    r2_scores_mlp.append(fold_r2_mlp)

# Calculate mean and standard deviation of regression metrics for MLP Regressor
mean_mse_mlp = pd.Series(mse_scores_mlp).mean()
std_mse_mlp = pd.Series(mse_scores_mlp).std()

mean_mae_mlp = pd.Series(mae_scores_mlp).mean()
std_mae_mlp = pd.Series(mae_scores_mlp).std()

mean_r2_mlp = pd.Series(r2_scores_mlp).mean()
std_r2_mlp = pd.Series(r2_scores_mlp).std()

print(f'Best Hyperparameters (MLP Regressor): {best_params_mlp}')
print(f'Mean Squared Error (MLP Regressor): {mean_mse_mlp:.4f} ± {std_mse_mlp:.4f}')
print(f'Mean Absolute Error (MLP Regressor): {mean_mae_mlp:.4f} ± {std_mae_mlp:.4f}')
print(f'R-squared (MLP Regressor): {mean_r2_mlp:.4f} ± {std_r2_mlp:.4f}')

# Use the best hyperparameters to train the final model on the entire training set for MLP Regressor
final_model_mlp = MLPRegressor(max_iter=1000, random_state=42, **best_params_mlp)
final_model_mlp.fit(X_train, y_train)

# Make predictions on the test set for MLP Regressor
y_pred_mlp = final_model_mlp.predict(X_test)


In [ ]:
from sklearn.model_selection import KFold, RandomizedSearchCV
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np


X = df.drop('Total_Score', axis=1)
y = df['Total_Score']

#  X and y are features and target variable
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# --- MLP Regressor ---

# Define the hyperparameter grid for randomized search
param_dist_mlp = {
    'hidden_layer_sizes': [(50,), (100,), (50, 50), (100, 50)],
    'activation': ['logistic', 'tanh', 'relu'],
    'alpha': np.logspace(-4, 0, 5),
}

# Initialize MLP Regressor
mlp_regressor = MLPRegressor(max_iter=1000, random_state=42)

# Initialize RandomizedSearchCV
random_search_mlp = RandomizedSearchCV(mlp_regressor, param_distributions=param_dist_mlp, n_iter=10, cv=5, scoring='neg_mean_squared_error', random_state=42, n_jobs=-1)

# Fit the model with hyperparameter tuning on the training set
random_search_mlp.fit(X_train, y_train)

# Get the best hyperparameters for MLP Regressor
best_params_mlp = random_search_mlp.best_params_

# Initialize KFold cross-validation for MLP Regressor
k_folds_mlp = 10
kf_mlp = KFold(n_splits=k_folds_mlp, shuffle=True, random_state=42)

# Initialize lists to store regression metrics for each fold
mse_scores_mlp = []
mae_scores_mlp = []
r2_scores_mlp = []
mape_scores_mlp = []
mspe_scores_mlp = []

# Perform k-fold cross-validation for MLP Regressor
for fold, (train_index, val_index) in enumerate(kf_mlp.split(X_train)):
    X_train_fold_mlp, X_val_fold_mlp = X_train.iloc[train_index], X_train.iloc[val_index]
    y_train_fold_mlp, y_val_fold_mlp = y_train.iloc[train_index], y_train.iloc[val_index]

    # Initialize MLP Regressor with the best hyperparameters
    final_model_mlp = MLPRegressor(max_iter=1000, random_state=42, **best_params_mlp)

    # Fit the model to the training fold
    final_model_mlp.fit(X_train_fold_mlp, y_train_fold_mlp)

    # Make predictions on the validation fold
    y_val_pred_mlp = final_model_mlp.predict(X_val_fold_mlp)

    # Calculate regression metrics for the fold
    fold_mse_mlp = mean_squared_error(y_val_fold_mlp, y_val_pred_mlp)
    fold_mae_mlp = mean_absolute_error(y_val_fold_mlp, y_val_pred_mlp)
    fold_r2_mlp = r2_score(y_val_fold_mlp, y_val_pred_mlp)

    # Calculate MAPE
    fold_mape_mlp = np.mean(np.abs((y_val_fold_mlp - y_val_pred_mlp) / y_val_fold_mlp)) * 100

    # Calculate MSPE
    fold_mspe_mlp = np.mean(((y_val_fold_mlp - y_val_pred_mlp) / y_val_fold_mlp) ** 2) * 100

    mse_scores_mlp.append(fold_mse_mlp)
    mae_scores_mlp.append(fold_mae_mlp)
    r2_scores_mlp.append(fold_r2_mlp)
    mape_scores_mlp.append(fold_mape_mlp)
    mspe_scores_mlp.append(fold_mspe_mlp)

    print(f'Mean Squared Error for Fold {fold + 1}: {fold_mse_mlp}')
    print(f'Mean Absolute Error for Fold {fold + 1}: {fold_mae_mlp}')
    print(f'R-squared for Fold {fold + 1}: {fold_r2_mlp}')
    print(f'MAPE for Fold {fold + 1}: {fold_mape_mlp}%')
    print(f'MSPE for Fold {fold + 1}: {fold_mspe_mlp}%')

# Calculate mean and standard deviation of regression metrics for MLP Regressor
mean_mse_mlp = pd.Series(mse_scores_mlp).mean()
std_mse_mlp = pd.Series(mse_scores_mlp).std()

mean_mae_mlp = pd.Series(mae_scores_mlp).mean()
std_mae_mlp = pd.Series(mae_scores_mlp).std()

mean_r2_mlp = pd.Series(r2_scores_mlp).mean()
std_r2_mlp = pd.Series(r2_scores_mlp).std()

mean_mape_mlp = pd.Series(mape_scores_mlp).mean()
mean_mspe_mlp = pd.Series(mspe_scores_mlp).mean()

print(f'Best Hyperparameters (MLP Regressor): {best_params_mlp}')
print(f'Mean Squared Error (MLP Regressor): {mean_mse_mlp:.4f} ± {std_mse_mlp:.4f}')
print(f'Mean Absolute Error (MLP Regressor): {mean_mae_mlp:.4f} ± {std_mae_mlp:.4f}')
print(f'R-squared (MLP Regressor): {mean_r2_mlp:.4f} ± {std_r2_mlp:.4f}')
print(f'MAPE (MLP Regressor): {mean_mape_mlp:.4f}%')
print(f'MSPE (MLP Regressor): {mean_mspe_mlp:.4f}%')

# Use the best hyperparameters to train the final model on the entire training set for MLP Regressor
final_model_mlp = MLPRegressor(max_iter=1000, random_state=42, **best_params_mlp)
final_model_mlp.fit(X_train, y_train)

# Make predictions on the test set for MLP Regressor
y_pred_mlp = final_model_mlp.predict(X_test)

# Evaluate the model on the test set for MLP Regressor
test_mse_mlp = mean_squared_error(y_test, y_pred_mlp)
test_mae_mlp = mean_absolute_error(y_test, y_pred_mlp)
test_r2_mlp = r2_score(y_test, y_pred_mlp)

# Calculate MAPE for the test set for MLP Regressor
test_mape_mlp = np.mean(np.abs((y_test - y_pred_mlp) / y_test)) * 100

# Calculate MSPE for the test set for MLP Regressor
test_mspe_mlp = np.mean(((y_test - y_pred_mlp) / y_test) ** 2) * 100

print(f'Best Hyperparameters (MLP Regressor): {best_params_mlp}')
print(f'Mean Squared Error on Test Set (MLP Regressor): {test_mse_mlp:.4f}')
print(f'Mean Absolute Error on Test Set (MLP Regressor): {test_mae_mlp:.4f}')
print(f'R-squared on Test Set (MLP Regressor): {test_r2_mlp:.4f}')
print(f'MAPE on Test Set (MLP Regressor): {test_mape_mlp:.4f}%')
print(f'MSPE on Test Set (MLP Regressor): {test_mspe_mlp:.4f}%')


In [ ]:
# Optional: Plot predicted vs. actual values for better visualization
y_pred = final_model.predict(X_test)
plt.scatter(y_test, y_pred_mlp)
plt.xlabel("Actual Values")
plt.ylabel("Predicted Values")
plt.title(" poission Regressor - Predicted vs. Actual Values")
plt.show()

## Classification(satisfaction)

###Naive Bayes
####GaussianNB

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


# Separate features and target variable
X = df1.drop('satisfaction', axis=1)
y = df1['satisfaction']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the Gaussian Naive Bayes classifier
nb_classifier = GaussianNB()

# Train the model on the training data
nb_classifier.fit(X_train, y_train)

# Make predictions on the test data
y_pred = nb_classifier.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

# Print the evaluation metrics
print(f'Accuracy: {accuracy}')
print(f'Confusion Matrix:\n{conf_matrix}')
print(f'Classification Report:\n{classification_rep}')


In [ ]:
# Separate features and target variable
X = dfc.drop('satisfaction', axis=1)
y = dfc['satisfaction']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the Gaussian Naive Bayes classifier
nb_classifier = GaussianNB()

# Train the model on the training data
nb_classifier.fit(X_train, y_train)

# Make predictions on the test data
y_pred = nb_classifier.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

# Print the evaluation metrics
print(f'Accuracy: {accuracy}')
print(f'Confusion Matrix:\n{conf_matrix}')
print(f'Classification Report:\n{classification_rep}')


In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix

# Given confusion matrix
conf_matrix = np.array([[11493, 2676], [2496, 11688]])

# Extract values from confusion matrix
tn, fp, fn, tp = conf_matrix.ravel()

# Calculate evaluation metrics
accuracy = (tp + tn) / (tp + tn + fp + fn)
precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0
f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0

# Calculate Sensitivity (True Positive Rate or Recall for the positive class)
sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0.0

# Calculate Specificity (True Negative Rate)
specificity = tn / (tn + fp) if (tn + fp) > 0 else 0.0

# Print evaluation metrics
print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")
print(f"Sensitivity: {sensitivity:.2f}")
print(f"Specificity: {specificity:.2f}")


In [ ]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

# Make predictions on the test data
y_pred_proba = nb_classifier.predict_proba(X_test)[:, 1]

# Compute ROC curve and AUC
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
roc_auc = auc(fpr, tpr)


# Plot the ROC curve
plt.figure(figsize=(10, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random Guess')

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.show()


KNN

In [ ]:
#scalling process specified for df1
c = df1.columns

from sklearn.preprocessing import MinMaxScaler

Scaler = MinMaxScaler()
df1 = Scaler.fit_transform(df1)

df1 = pd.DataFrame(df1)

df1.columns = c

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import numpy as np


X = df1.drop('satisfaction', axis=1)
y = df1['satisfaction']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the KNN model
knn_model = KNeighborsClassifier()

# Define the hyperparameter grid
param_grid = {
    'n_neighbors': [3, 5, 7, 9],  
    'weights': ['uniform', 'distance'],
    'p': [1, 2]  # 1 for Manhattan distance, 2 for Euclidean distance
}

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=knn_model, param_grid=param_grid, scoring='accuracy', cv=5, n_jobs=-1)

# Fit the model with hyperparameter tuning on the training set
grid_search.fit(X_train, y_train)

# Get the best hyperparameters
best_params = grid_search.best_params_

# Initialize KFold cross-validation
k_folds = 10
kf = KFold(n_splits=k_folds, shuffle=True, random_state=42)

# Initialize an array to store accuracy scores for each fold
accuracy_scores = []

# Perform k-fold cross-validation
for train_index, test_index in kf.split(X):
    X_train_fold, X_val_fold = X.iloc[train_index], X.iloc[test_index]
    y_train_fold, y_val_fold = y.iloc[train_index], y.iloc[test_index]

    # Use the best hyperparameters to train the final model
    final_model = KNeighborsClassifier(**best_params)
    final_model.fit(X_train_fold, y_train_fold)

    # Make predictions on the validation set
    y_val_pred = final_model.predict(X_val_fold)

    # Calculate accuracy for the fold
    fold_accuracy = accuracy_score(y_val_fold, y_val_pred)
    accuracy_scores.append(fold_accuracy)

# Calculate mean and standard deviation of accuracy scores
mean_accuracy = np.mean(accuracy_scores)
std_accuracy = np.std(accuracy_scores)

print(f'Best Hyperparameters: {best_params}')
print(f'Mean Accuracy: {mean_accuracy}')
print(f'Standard Deviation of Accuracy: {std_accuracy}')

# Use the best hyperparameters to train the final model on the entire training set
final_model = KNeighborsClassifier(**best_params)
final_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = final_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

print(f'Accuracy on Test Set: {accuracy}')
print(f'Confusion Matrix:\n{conf_matrix}')
print(f'Classification Report:\n{classification_rep}')


Best Hyperparameters: {'n_neighbors': 7, 'p': 1, 'weights': 'distance'}

Accuracy: 0.9332380878832343

Confusion Matrix:

[[11104   717]

 [ 1012 13065]]

Classification Report:

              precision    recall  f1-score   support

         0.0       0.92      0.94      0.93     11821
         1.0       0.95      0.93      0.94     14077

      accuracy                           0.93     25898
      macro avg      0.93      0.93      0.93     25898
      weighted avg   0.93      0.93      0.93     25898

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix

# Given confusion matrix
conf_matrix = np.array([[11104, 717], [1012, 13065]])

# Extract values from confusion matrix
tn, fp, fn, tp = conf_matrix.ravel()

# Calculate evaluation metrics
accuracy = (tp + tn) / (tp + tn + fp + fn)
precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0
f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0

# Calculate Sensitivity (True Positive Rate or Recall for the positive class)
sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0.0

# Calculate Specificity (True Negative Rate)
specificity = tn / (tn + fp) if (tn + fp) > 0 else 0.0

# Print evaluation metrics
print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")
print(f"Sensitivity: {sensitivity:.2f}")
print(f"Specificity: {specificity:.2f}")


In [ ]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

# Get predicted probabilities for the positive class
y_test_pred_proba = final_model.predict_proba(X_test)[:, 1]

# Compute ROC curve and AUC
fpr, tpr, thresholds = roc_curve(y_test, y_test_pred_proba)
roc_auc = auc(fpr, tpr)

# Plot the ROC curve
plt.figure(figsize=(10, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random Guess')

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.show()


SVM

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import numpy as np

# dfc is my scaled and balanced dataframe
X = dfc.drop('satisfaction', axis=1)
y = dfc['satisfaction']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the SVM model
svm_model = SVC()

# Define the hyperparameter grid
param_grid = {
    'C': [0.1, 1, 10],  
    'kernel': ['linear', 'rbf', 'poly'],
    'gamma': ['scale', 'auto']
}

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=svm_model, param_grid=param_grid, scoring='accuracy', cv=5, n_jobs=-1)

# Fit the model with hyperparameter tuning on the training set
grid_search.fit(X_train, y_train)

# Get the best hyperparameters
best_params = grid_search.best_params_

# Use the best hyperparameters to train the final model
final_model = SVC(**best_params, probability=True)
final_model.fit(X_train, y_train)


# Initialize KFold cross-validation
k_folds = 10
kf = KFold(n_splits=k_folds, shuffle=True, random_state=42)

# Initialize an array to store accuracy scores for each fold
accuracy_scores = []

# Initialize arrays to store predictions and true labels for the test set
y_test_pred = np.array([])
y_test_true = np.array([])

# Perform k-fold cross-validation
for train_index, test_index in kf.split(X):
    X_train_fold, X_val_fold = X.iloc[train_index], X.iloc[test_index]
    y_train_fold, y_val_fold = y.iloc[train_index], y.iloc[test_index]

    # Use the best hyperparameters to train the final model
    final_model = SVC(**best_params)
    final_model.fit(X_train_fold, y_train_fold)

    # Make predictions on the validation set
    y_val_pred = final_model.predict(X_val_fold)

    # Calculate accuracy for the fold
    fold_accuracy = accuracy_score(y_val_fold, y_val_pred)
    accuracy_scores.append(fold_accuracy)

    # Store predictions and true labels for the test set
    y_test_pred = np.concatenate((y_test_pred, final_model.predict(X_test.iloc[test_index])))
    y_test_true = np.concatenate((y_test_true, y_test.iloc[test_index]))

# Calculate mean and standard deviation of accuracy scores
mean_accuracy = np.mean(accuracy_scores)
std_accuracy = np.std(accuracy_scores)

print(f'Best Hyperparameters: {best_params}')
print(f'Mean Accuracy: {mean_accuracy}')
print(f'Standard Deviation of Accuracy: {std_accuracy}')

# Evaluate the model on the test set
accuracy = accuracy_score(y_test_true, y_test_pred)
conf_matrix = confusion_matrix(y_test_true, y_test_pred)
classification_rep = classification_report(y_test_true, y_test_pred)

print(f'Accuracy on Test Set: {accuracy}')
print(f'Confusion Matrix:\n{conf_matrix}')
print(f'Classification Report:\n{classification_rep}')


Best Hyperparameters: {'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}

Accuracy: 0.9534088103551652

Confusion Matrix:

[[13651 518]

 [ 803 13381]]

Classification Report:

                  precision  recall   f1-score   support
        0.0       0.94       0.96       0.95     14169
        1.0       0.96       0.94       0.95     14184

     accuracy                           0.95     28353
     macro avg    0.95       0.95       0.95     28353
     weighted avg 0.95       0.95       0.95     28353

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix

# Given confusion matrix
conf_matrix = np.array([[13651, 518], [803, 13381]])

# Extract values from confusion matrix
tn, fp, fn, tp = conf_matrix.ravel()

# Calculate evaluation metrics
accuracy = (tp + tn) / (tp + tn + fp + fn)
precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0
f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0

# Calculate Sensitivity (True Positive Rate or Recall for the positive class)
sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0.0

# Calculate Specificity (True Negative Rate)
specificity = tn / (tn + fp) if (tn + fp) > 0 else 0.0

# Print evaluation metrics
print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")
print(f"Sensitivity: {sensitivity:.2f}")
print(f"Specificity: {specificity:.2f}")


In [ ]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

# Get predicted probabilities for the positive class
y_test_pred_proba = final_model.predict_proba(X_test)[:, 1]

# Compute ROC curve and AUC
fpr, tpr, thresholds = roc_curve(y_test, y_test_pred_proba)
roc_auc = auc(fpr, tpr)

# Plot the ROC curve
plt.figure(figsize=(10, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random Guess')

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.show()


DT

In [ ]:
# df2 is my SMOTE applied Dataframe
X = df2.drop(columns = ['satisfaction'])
y = df2['satisfaction']

#Using SMOTE to balance the Data
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state = 42)
X, y = sm.fit_resample(X, y)
pd.Series(y).value_counts()
df2 = X.join(y)

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import numpy as np

# df2 is SMOTE
X = df2.drop('satisfaction', axis=1)
y = df2['satisfaction']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the Decision Tree model
dt_model = DecisionTreeClassifier()

# Define the hyperparameter grid
param_grid = {
    'criterion': ['gini', 'entropy'],
    'splitter': ['best', 'random'],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=dt_model, param_grid=param_grid, scoring='accuracy', cv=5, n_jobs=-1)

# Fit the model with hyperparameter tuning on the training set
grid_search.fit(X_train, y_train)

# Get the best hyperparameters
best_params = grid_search.best_params_

# Initialize KFold cross-validation
k_folds = 10
kf = KFold(n_splits=k_folds, shuffle=True, random_state=42)

# Initialize an array to store accuracy scores for each fold
accuracy_scores = []

# Perform k-fold cross-validation
for train_index, test_index in kf.split(X):
    X_train_fold, X_val_fold = X.iloc[train_index], X.iloc[test_index]
    y_train_fold, y_val_fold = y.iloc[train_index], y.iloc[test_index]

    # Use the best hyperparameters to train the final model
    final_model = DecisionTreeClassifier(**best_params)
    final_model.fit(X_train_fold, y_train_fold)

    # Make predictions on the validation set
    y_val_pred = final_model.predict(X_val_fold)

    # Calculate accuracy for the fold
    fold_accuracy = accuracy_score(y_val_fold, y_val_pred)
    accuracy_scores.append(fold_accuracy)

# Calculate mean and standard deviation of accuracy scores
mean_accuracy = np.mean(accuracy_scores)
std_accuracy = np.std(accuracy_scores)

print(f'Best Hyperparameters: {best_params}')
print(f'Mean Accuracy: {mean_accuracy}')
print(f'Standard Deviation of Accuracy: {std_accuracy}')

# Use the best hyperparameters to train the final model on the entire training set
final_model = DecisionTreeClassifier(**best_params)
final_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = final_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

print(f'Accuracy on Test Set: {accuracy}')
print(f'Confusion Matrix:\n{conf_matrix}')
print(f'Classification Report:\n{classification_rep}')


Best Hyperparameters: {'criterion': 'gini', 'max_depth': 20, 'min_samples_leaf': 1, 'min_samples_split': 10, 'splitter': 'best'}

Accuracy: 0.9444150530808028

Confusion Matrix:

[[13505   664]

 [  912 13272]]


Classification Report:

              precision    recall  f1-score   support

           0       0.94      0.95      0.94     14169
           1       0.95      0.94      0.94     14184

    accuracy                           0.94     28353
    macro avg       0.94      0.94     0.94     28353
    weighted avg    0.94      0.94     0.94     28353

In [ ]:
from sklearn.tree import DecisionTreeClassifier, plot_tree
import matplotlib.pyplot as plt

#  X_train and y_train are training data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the Decision Tree model with the best hyperparameters
best_params = {'criterion': 'gini', 'max_depth': 20, 'min_samples_leaf': 1, 'min_samples_split': 10, 'splitter': 'best'}
final_model = DecisionTreeClassifier(**best_params)

# Train the final model
final_model.fit(X_train, y_train)

# Visualize the Decision Tree
plt.figure(figsize=(15, 10))
plot_tree(final_model, filled=True, feature_names=X_train.columns, class_names=['Not Satisfied', 'Satisfied'])
plt.title('Decision Tree Visualization')
plt.show()

In [ ]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

# Get predicted probabilities for the positive class
y_test_pred_proba = grid_search.predict_proba(X_test)[:, 1]

# Compute ROC curve and AUC
fpr, tpr, thresholds = roc_curve(y_test, y_test_pred_proba)
roc_auc = auc(fpr, tpr)

# Plot the ROC curve
plt.figure(figsize=(10, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random Guess')

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.show()


In [ ]:
# Plot feature importances
feature_importances = final_model.feature_importances_
sorted_idx = np.argsort(feature_importances)[::-1]

# Plot the 10 most important features
top_features = 10
plt.figure(figsize=(10, 6))
plt.bar(range(top_features), feature_importances[sorted_idx][:top_features], align="center")
plt.xticks(range(top_features), X_train.columns[sorted_idx][:top_features], rotation=45)
plt.xlabel("Feature")
plt.ylabel("Feature Importance")
plt.title("Top 10 Feature Importances in DT Model")
plt.show()

GB

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV, KFold
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import numpy as np

#  dfq is  DataFrame after SMOTE
X = df2.drop('satisfaction', axis=1)
y = df2['satisfaction']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the Gradient Boosting model
gb_model = GradientBoostingClassifier()

# Define the hyperparameter grid
param_dist = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Initialize RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=gb_model,
    param_distributions=param_dist,
    n_iter=20,
    scoring='accuracy',
    cv=5,
    n_jobs=-1,
    random_state=42
)

# Fit the model with hyperparameter tuning on the training set
random_search.fit(X_train, y_train)

# Get the best hyperparameters
best_params = random_search.best_params_

# Initialize KFold cross-validation
k_folds = 10
kf = KFold(n_splits=k_folds, shuffle=True, random_state=42)

# Initialize an array to store accuracy scores for each fold
accuracy_scores = []

# Perform k-fold cross-validation
for train_index, test_index in kf.split(X):
    X_train_fold, X_val_fold = X.iloc[train_index], X.iloc[test_index]
    y_train_fold, y_val_fold = y.iloc[train_index], y.iloc[test_index]

    # Use the best hyperparameters to train the final model
    final_model = GradientBoostingClassifier(**best_params)
    final_model.fit(X_train_fold, y_train_fold)

    # Make predictions on the validation set
    y_val_pred = final_model.predict(X_val_fold)

    # Calculate accuracy for the fold
    fold_accuracy = accuracy_score(y_val_fold, y_val_pred)
    accuracy_scores.append(fold_accuracy)

# Calculate mean and standard deviation of accuracy scores
mean_accuracy = np.mean(accuracy_scores)
std_accuracy = np.std(accuracy_scores)

print(f'Best Hyperparameters: {best_params}')
print(f'Mean Accuracy: {mean_accuracy}')
print(f'Standard Deviation of Accuracy: {std_accuracy}')

# Use the best hyperparameters to train the final model on the entire training set
final_model = GradientBoostingClassifier(**best_params)
final_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = final_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

print(f'Accuracy on Test Set: {accuracy}')
print(f'Confusion Matrix:\n{conf_matrix}')
print(f'Classification Report:\n{classification_rep}')


Best Hyperparameters: {'learning_rate': 0.1, 'max_depth': 7,
'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 200}

Accuracy: 0.9608154339928755

Confusion Matrix:

[[13758 411]

[ 700 13484]]

Classification Report:

                     precision       recall       f1-score      support

             0        0.95            0.97         0.96          14169
             1        0.97            0.95         0.96          14184

       accuracy                                  0.96          28353
       macro avg     0.96           0.96         0.96          28353
       weighted avg  0.96           0.96         0.96          28353

In [ ]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

# Get predicted probabilities for the positive class
y_test_pred_proba = random_search.predict_proba(X_test)[:, 1]

# Compute ROC curve and AUC
fpr, tpr, thresholds = roc_curve(y_test, y_test_pred_proba)
roc_auc = auc(fpr, tpr)

# Plot the ROC curve
plt.figure(figsize=(10, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random Guess')

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.show()


RF

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import numpy as np

#  df2 is  DataFrame after SMOTE
X = df2.drop('satisfaction', axis=1)
y = df2['satisfaction']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the Random Forest model
rf_model = RandomForestClassifier()

# Define the hyperparameter grid
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, scoring='accuracy', cv=5, n_jobs=-1)

# Fit the model with hyperparameter tuning on the training set
grid_search.fit(X_train, y_train)

# Get the best hyperparameters
best_params = grid_search.best_params_

# Initialize KFold cross-validation
k_folds = 10
kf = KFold(n_splits=k_folds, shuffle=True, random_state=42)

# Initialize an array to store accuracy scores for each fold
accuracy_scores = []

# Perform k-fold cross-validation
for train_index, test_index in kf.split(X):
    X_train_fold, X_val_fold = X.iloc[train_index], X.iloc[test_index]
    y_train_fold, y_val_fold = y.iloc[train_index], y.iloc[test_index]

    # Use the best hyperparameters to train the final model
    final_model = RandomForestClassifier(**best_params)
    final_model.fit(X_train_fold, y_train_fold)

    # Make predictions on the validation set
    y_val_pred = final_model.predict(X_val_fold)

    # Calculate accuracy for the fold
    fold_accuracy = accuracy_score(y_val_fold, y_val_pred)
    accuracy_scores.append(fold_accuracy)

# Calculate mean and standard deviation of accuracy scores
mean_accuracy = np.mean(accuracy_scores)
std_accuracy = np.std(accuracy_scores)

print(f'Best Hyperparameters: {best_params}')
print(f'Mean Accuracy: {mean_accuracy}')
print(f'Standard Deviation of Accuracy: {std_accuracy}')

# Use the best hyperparameters to train the final model on the entire training set
final_model = RandomForestClassifier(**best_params)
final_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = final_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

print(f'Accuracy on Test Set: {accuracy}')
print(f'Confusion Matrix:\n{conf_matrix}')
print(f'Classification Report:\n{classification_rep}')


Best Hyperparameters: {'bootstrap': False, 'max_depth': 30,
'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 200}

Accuracy: 0.9613092089020562

 Confusion Matrix:

  [[13776 393]

   [ 704 13480]]

Classification Report:

                        precision      recall     f1-score      support
            0             0.95          0.97        0.96        14169
            1             0.97         0.95         0.96        14184
            
     accuracy                                       0.96        28353
     macroavg             0.96         0.96         0.96        28353
     weighted avg         0.96         0.96         0.96        28353


In [ ]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

# Get predicted probabilities for the positive class
y_test_pred_proba = grid_search.predict_proba(X_test)[:, 1]

# Compute ROC curve and AUC
fpr, tpr, thresholds = roc_curve(y_test, y_test_pred_proba)
roc_auc = auc(fpr, tpr)

# Plot the ROC curve
plt.figure(figsize=(10, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random Guess')

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.show()


In [ ]:
# Plot feature importances
feature_importances = final_model.feature_importances_
sorted_idx = np.argsort(feature_importances)[::-1]

# Plot the 10 most important features
top_features = 10
plt.figure(figsize=(10, 6))
plt.bar(range(top_features), feature_importances[sorted_idx][:top_features], align="center")
plt.xticks(range(top_features), X_train.columns[sorted_idx][:top_features], rotation=45)
plt.xlabel("Feature")
plt.ylabel("Feature Importance")
plt.title("Top 10 Feature Importances in Random Forest Model")
plt.show()

AdaBoost

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
import pandas as pd

#  df2 is SMOTE applied df
X = df2.drop('satisfaction', axis=1) 
y = df2['satisfaction']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the AdaBoost model
adaboost_model = AdaBoostClassifier()

# Define the hyperparameter grid
param_grid = {
    'n_estimators': [50, 100, 150],
    'learning_rate': [0.01, 0.1, 1.0]
}

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=adaboost_model, param_grid=param_grid, scoring='accuracy', cv=5)

# Fit the model to the training data
grid_search.fit(X_train, y_train)

# Get the best hyperparameters
best_params = grid_search.best_params_
best_n_estimators = best_params['n_estimators']
best_learning_rate = best_params['learning_rate']

# Initialize KFold cross-validation
k_folds = 10
kf = KFold(n_splits=k_folds, shuffle=True, random_state=42)

# Initialize an array to store accuracy scores for each fold
accuracy_scores = []

# Perform k-fold cross-validation
for train_index, val_index in kf.split(X):
    X_train_fold, X_val_fold = X.iloc[train_index], X.iloc[val_index]
    y_train_fold, y_val_fold = y.iloc[train_index], y.iloc[val_index]

    # Initialize AdaBoost model with the best hyperparameters
    final_model = AdaBoostClassifier(n_estimators=best_n_estimators, learning_rate=best_learning_rate)

    # Fit the model to the training fold
    final_model.fit(X_train_fold, y_train_fold)

    # Make predictions on the validation fold
    y_val_pred = final_model.predict(X_val_fold)

    # Calculate accuracy for the fold
    fold_accuracy = accuracy_score(y_val_fold, y_val_pred)

    accuracy_scores.append(fold_accuracy)

# Calculate mean and standard deviation of accuracy scores
mean_accuracy = pd.Series(accuracy_scores).mean()
std_accuracy = pd.Series(accuracy_scores).std()

# Print the best hyperparameters and evaluation metric
print(f"Best Number of Estimators: {best_n_estimators}")
print(f"Best Learning Rate: {best_learning_rate}")
print(f"Mean Accuracy: {mean_accuracy:.4f}")
print(f"Standard Deviation of Accuracy: {std_accuracy:.4f}")
print("\n")

# Evaluate on the test set
y_test_pred = grid_search.predict(X_test)

# Print additional metrics on the test set
print("Test Set Metrics:")
print("Accuracy:", accuracy_score(y_test, y_test_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_test_pred))
print("Classification Report:\n", classification_report(y_test, y_test_pred))


In [ ]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

# Get predicted probabilities for the positive class
y_test_pred_proba = grid_search.predict_proba(X_test)[:, 1]

# Compute ROC curve and AUC
fpr, tpr, thresholds = roc_curve(y_test, y_test_pred_proba)
roc_auc = auc(fpr, tpr)

# Plot ROC curve
plt.figure(figsize=(10, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random Guess')

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.show()


CatBoost

In [ ]:
from catboost import CatBoostClassifier
from sklearn.model_selection import GridSearchCV, train_test_split, KFold
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import pandas as pd

# df is not balanced or scaled
X = df.drop('satisfaction', axis=1)
y = df['satisfaction']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the CatBoost model
catboost_model = CatBoostClassifier()

# Define the hyperparameter grid
param_grid = {
    'iterations': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],  # Fix the typo here
    'depth': [6, 8, 10],
    'l2_leaf_reg': [1, 3, 5]
}

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=catboost_model, param_grid=param_grid, scoring='accuracy', cv=5)

# Fit the model to the training data
grid_search.fit(X_train, y_train, cat_features=None, verbose=0)

# Get the best hyperparameters
best_params = grid_search.best_params_
best_iterations = best_params['iterations']
best_learning_rate = best_params['learning_rate']
best_depth = best_params['depth']
best_l2_leaf_reg = best_params['l2_leaf_reg']

# Print the best hyperparameters
print(f"Best Iterations: {best_iterations}")
print(f"Best Learning Rate: {best_learning_rate}")
print(f"Best Depth: {best_depth}")
print(f"Best L2 Leaf Regularization: {best_l2_leaf_reg}")
print("\n")

# Initialize KFold cross-validation
k_folds = 10
kf = KFold(n_splits=k_folds, shuffle=True, random_state=42)

# Initialize an array to store accuracy scores for each fold
accuracy_scores = []

# Perform k-fold cross-validation
for fold, (train_index, val_index) in enumerate(kf.split(X_train)):
    X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]
    y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]

    # Initialize CatBoost model with the best hyperparameters
    final_model = CatBoostClassifier(
        iterations=best_iterations,
        learning_rate=best_learning_rate,
        depth=best_depth,
        l2_leaf_reg=best_l2_leaf_reg
    )

    # Fit the model to the training fold
    final_model.fit(X_train_fold, y_train_fold, cat_features=None, verbose=0)

    # Make predictions on the validation fold
    y_val_pred = final_model.predict(X_val_fold)

    # Calculate accuracy for the fold
    fold_accuracy = accuracy_score(y_val_fold, y_val_pred)
    accuracy_scores.append(fold_accuracy)

    print(f"Fold {fold + 1}: Accuracy = {fold_accuracy:.4f}")

# Calculate mean and standard deviation of accuracy scores
mean_accuracy = pd.Series(accuracy_scores).mean()
std_accuracy = pd.Series(accuracy_scores).std()

# Print the mean and standard deviation of accuracy scores
print(f"\nMean Accuracy: {mean_accuracy:.4f}")
print(f"Standard Deviation of Accuracy: {std_accuracy:.4f}")
print("\n")

# Evaluate on the test set
y_test_pred = grid_search.predict(X_test)
print("Test Set Metrics:")
print("Accuracy:", accuracy_score(y_test, y_test_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_test_pred))
print("Classification Report:\n", classification_report(y_test, y_test_pred))


In [ ]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

# Get predicted probabilities for the positive class
y_test_pred_proba = final_model.predict_proba(X_test)[:, 1]

# Compute ROC curve and AUC
fpr, tpr, thresholds = roc_curve(y_test, y_test_pred_proba)
roc_auc = auc(fpr, tpr)


# Plot the ROC curve
plt.figure(figsize=(10, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random Guess')

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.show()


In [ ]:
# Given confusion matrix
conf_matrix = np.array([[11430, 391], [623, 13454]])

# Extract values from confusion matrix
tn, fp, fn, tp = conf_matrix.ravel()

# Calculate evaluation metrics
accuracy = (tp + tn) / (tp + tn + fp + fn)
precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0
f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0

# Calculate Sensitivity (True Positive Rate or Recall for the positive class)
sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0.0

# Calculate Specificity (True Negative Rate)
specificity = tn / (tn + fp) if (tn + fp) > 0 else 0.0

# Print evaluation metrics
print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")
print(f"Sensitivity: {sensitivity:.2f}")
print(f"Specificity: {specificity:.2f}")


NN

In [ ]:
from sklearn.model_selection import KFold
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

#  dfq is  DataFrame after SMOTE
X = df.drop('satisfaction', axis=1)
y = df['satisfaction']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the Neural Network model
mlp_model = MLPClassifier(max_iter=1000, random_state=42)

# Define the hyperparameter grid
param_grid = {
    'hidden_layer_sizes': [(50,), (100,), (50, 50), (100, 50)],
    'activation': ['logistic', 'tanh', 'relu'],
    'alpha': [0.0001, 0.001, 0.01],
}

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=mlp_model, param_grid=param_grid, scoring='accuracy', cv=5, n_jobs=-1)

# Fit the model with hyperparameter tuning on the training set
grid_search.fit(X_train, y_train)

# Get the best hyperparameters
best_params = grid_search.best_params_

# Initialize KFold cross-validation
k_folds = 10
kf = KFold(n_splits=k_folds, shuffle=True, random_state=42)

# Initialize an array to store accuracy scores for each fold
accuracy_scores = []

# Perform k-fold cross-validation
for train_index, test_index in kf.split(X):
    X_train_fold, X_val_fold = X.iloc[train_index], X.iloc[test_index]
    y_train_fold, y_val_fold = y.iloc[train_index], y.iloc[test_index]

    # Use the best hyperparameters to train the final model
    final_model = MLPClassifier(max_iter=1000, random_state=42, **best_params)
    final_model.fit(X_train_fold, y_train_fold)

    # Make predictions on the validation set
    y_val_pred = final_model.predict(X_val_fold)

    # Calculate accuracy for the fold
    fold_accuracy = accuracy_score(y_val_fold, y_val_pred)
    accuracy_scores.append(fold_accuracy)

# Calculate mean and standard deviation of accuracy scores
mean_accuracy = np.mean(accuracy_scores)
std_accuracy = np.std(accuracy_scores)

print(f'Best Hyperparameters: {best_params}')
print(f'Mean Accuracy: {mean_accuracy}')
print(f'Standard Deviation of Accuracy: {std_accuracy}')

# Use the best hyperparameters to train the final model on the entire training set
final_model = MLPClassifier(max_iter=1000, random_state=42, **best_params)
final_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = final_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

print(f'Accuracy on Test Set: {accuracy}')
print(f'Confusion Matrix:\n{conf_matrix}')
print(f'Classification Report:\n{classification_rep}')

In [ ]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

#  dfq is  DataFrame after SMOTE
X = df.drop('satisfaction', axis=1)
y = df['satisfaction']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the Neural Network model with the best hyperparameters
best_params = {'activation': 'logistic', 'alpha': 0.001, 'hidden_layer_sizes': (100,)}
mlp_model = MLPClassifier(max_iter=1000, random_state=42, **best_params)

# Fit the model on the training set
mlp_model.fit(X_train, y_train)

# Generate predicted probabilities for the test set
y_val_prob = mlp_model.predict_proba(X_test)[:, 1]

#  y_test and y_pred are  true labels and predicted probabilities
fpr, tpr, thresholds = roc_curve(y_test, y_val_prob)

# Calculate the AUC score
roc_auc = auc(fpr, tpr)

# Plot the ROC curve
plt.figure(figsize=(10, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random Guess')

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.show()


In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import RocCurveDisplay, auc
from sklearn.model_selection import KFold
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

#  dfq is  DataFrame after SMOTE
X = df.drop('satisfaction', axis=1)
y = df['satisfaction']

# Define the Neural Network model with the best hyperparameters
best_params = {'activation': 'logistic', 'alpha': 0.001, 'hidden_layer_sizes': (100,)}
mlp_model = MLPClassifier(max_iter=1000, random_state=42, **best_params)

# Initialize KFold cross-validation
k_folds = 10
kf = KFold(n_splits=k_folds, shuffle=True, random_state=42)

# Initialize arrays to store true positive rates and area under the ROC curve for each fold
tprs = []
aucs = []

# Initialize mean false positive rate array
mean_fpr = np.linspace(0, 1, 100)

# Initialize the plot
fig, ax = plt.subplots(figsize=(8, 8))

# Perform k-fold cross-validation
for fold, (train_index, test_index) in enumerate(kf.split(X)):
    X_train_fold, X_val_fold = X.iloc[train_index], X.iloc[test_index]
    y_train_fold, y_val_fold = y.iloc[train_index], y.iloc[test_index]

    # Train the final model
    mlp_model.fit(X_train_fold, y_train_fold)

    # Get predicted probabilities for positive class
    y_val_prob = mlp_model.predict_proba(X_val_fold)[:, 1]

    # Compute ROC curve and area under the curve
    fpr, tpr, _ = roc_curve(y_val_fold, y_val_prob)
    roc_auc = auc(fpr, tpr)

    # Plot ROC curve for each fold
    roc_display = RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=roc_auc, estimator_name=f"Fold {fold}")
    roc_display.plot(ax=ax, alpha=0.3, lw=1)

    # Interpolate true positive rates to mean false positive rate
    interp_tpr = np.interp(mean_fpr, fpr, tpr)
    interp_tpr[0] = 0.0
    tprs.append(interp_tpr)
    aucs.append(roc_auc)

# Plot mean ROC curve
mean_tpr = np.mean(tprs, axis=0)
mean_tpr[-1] = 1.0
mean_auc = auc(mean_fpr, mean_tpr)

ax.plot(
    mean_fpr,
    mean_tpr,
    color="b",
    label=f"Mean ROC (AUC = {mean_auc:.2f})",
    lw=2,
    alpha=0.8,
)

# Plot chance level
ax.plot([0, 1], [0, 1], linestyle="--", color="grey", lw=2, label="Chance", alpha=0.8)

# Set plot properties
ax.set_xlim([0.0, 1.0])
ax.set_ylim([0.0, 1.05])
ax.set_xlabel("False Positive Rate")
ax.set_ylabel("True Positive Rate")
ax.set_title("ROC Curve with k-fold Cross-Validation")
ax.legend(loc="lower right")
plt.show()

# Train the final model on the entire dataset
mlp_model.fit(X, y)

# Get predictions on the entire dataset
y_pred = mlp_model.predict(X)

# Generate confusion matrix for the entire dataset
conf_mat = confusion_matrix(y, y_pred)
pring(conf_mat)

In [ ]:
# Given confusion matrix
conf_matrix = np.array([[11212, 609], [1641, 12436]])

# Extract values from confusion matrix
tn, fp, fn, tp = conf_matrix.ravel()

# Calculate evaluation metrics
accuracy = (tp + tn) / (tp + tn + fp + fn)
precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0
f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0

# Calculate Sensitivity (True Positive Rate or Recall for the positive class)
sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0.0

# Calculate Specificity (True Negative Rate)
specificity = tn / (tn + fp) if (tn + fp) > 0 else 0.0

# Print evaluation metrics
print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")
print(f"Sensitivity: {sensitivity:.2f}")
print(f"Specificity: {specificity:.2f}")


Logistic

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV, train_test_split, KFold
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix



X = dfc.drop('satisfaction', axis=1)
y = dfc['satisfaction']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the logistic regression model
logreg_model = LogisticRegression()

# Define the hyperparameter grid
param_dist = {
    'penalty': ['l1', 'l2', 'elasticnet', 'none'],
    'C': np.logspace(-4, 4, 20),
    'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
}

# Initialize RandomizedSearchCV
random_search = RandomizedSearchCV(logreg_model, param_distributions=param_dist, n_iter=20, cv=5, scoring='accuracy', verbose=1, n_jobs=-1, random_state=42)

# Fit the model with hyperparameter tuning on the training set
random_search.fit(X_train, y_train)

# Get the best hyperparameters
best_params = random_search.best_params_

# Initialize KFold cross-validation
k_folds = 10
kf = KFold(n_splits=k_folds, shuffle=True, random_state=42)

# Initialize an array to store accuracy for each fold
accuracy_scores = []

# Perform k-fold cross-validation
for train_index, val_index in kf.split(X_train):
    X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]
    y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]

    # Use the best hyperparameters to train the final model
    final_model = LogisticRegression(**best_params)
    final_model.fit(X_train_fold, y_train_fold)

    # Make predictions on the validation set
    y_val_pred = final_model.predict(X_val_fold)

    # Calculate accuracy for the fold
    fold_accuracy = accuracy_score(y_val_fold, y_val_pred)
    accuracy_scores.append(fold_accuracy)

# Calculate mean and standard deviation of accuracy scores
mean_accuracy = np.mean(accuracy_scores)
std_accuracy = np.std(accuracy_scores)

# Print the best hyperparameters and evaluation metrics
print(f'Best Hyperparameters: {best_params}')
print(f'Mean Accuracy: {mean_accuracy:.4f}')
print(f'Standard Deviation of Accuracy: {std_accuracy:.4f}')

# Use the best hyperparameters to train the final model on the entire training set
final_model = LogisticRegression(**best_params)
final_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = final_model.predict(X_test)

# Evaluate the model on the test set
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

print(f'Accuracy on Test Set: {accuracy:.4f}')
print(f'Confusion Matrix:\n{conf_matrix}')
print(f'Classification Report:\n{classification_rep}')

In [ ]:
from sklearn.metrics import roc_curve, auc

y_pred = final_model.predict_proba(X_test)[:, 1]
#  y_test and y_pred are  true labels and predicted probabilities
fpr, tpr, thresholds = roc_curve(y_test, y_pred)

# Calculate the AUC score
roc_auc = auc(fpr, tpr)

# Plot the ROC curve
plt.figure(figsize=(10, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random Guess')

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.show()

#Clustering

Kmeans

In [ ]:
# df1 is sclaed df
X = df1.drop('satisfaction', axis=1)

Number of clusters:


elbow method

In [ ]:
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

#  X is  data
distortions = []
K_range = range(2, 11)

for k in K_range:
    kmeanModel = KMeans(n_clusters=k)
    kmeanModel.fit(X)
    distortions.append(kmeanModel.inertia_)

plt.figure(figsize=(8, 6))
plt.plot(K_range, distortions, marker='o')
plt.title('Elbow Method for Optimal k')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Distortion (SSD)')
plt.show()


Silhouette Score

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

silhouette_scores = []
K_range = range(2, 11)

for k in K_range:
    kmeanModel = KMeans(n_clusters=k)
    kmeanModel.fit(X)
    labels = kmeanModel.labels_
    silhouette_scores.append(silhouette_score(X, labels))

plt.figure(figsize=(8, 6))
plt.plot(K_range, silhouette_scores, marker='o')
plt.title('Silhouette Score for Optimal k')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Silhouette Score')
plt.show()


Gap Statistics

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances
from gap_statistic import OptimalK

optimalK = OptimalK(parallel_backend='joblib')
n_clusters = optimalK(X, cluster_array=np.arange(1, 11))
print(f'Optimal number of clusters: {n_clusters}')


Davies-Bouldin Index

In [ ]:
from sklearn.metrics import davies_bouldin_score

K_range = range(2, 11)
db_scores = []

for k in K_range:
    kmeanModel = KMeans(n_clusters=k)
    kmeanModel.fit(X)
    labels = kmeanModel.labels_
    db_scores.append(davies_bouldin_score(X, labels))

plt.figure(figsize=(8, 6))
plt.plot(K_range, db_scores, marker='o')
plt.title('Davies-Bouldin Index for Optimal k')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Davies-Bouldin Index')
plt.show()

kmeans model

In [ ]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.metrics import davies_bouldin_score, silhouette_score, calinski_harabasz_score, adjusted_rand_score, jaccard_score
from sklearn.preprocessing import StandardScaler

## df1 is scaled df
X_scaled = df1.drop('satisfaction', axis=1)
y = df1['satisfaction']

# Set the number of clusters for k-means
n_clusters = 3

# Initialize k-means clustering with K-means++ initialization
kmeans = KMeans(n_clusters=n_clusters, init='k-means++', random_state=42)
y_pred = kmeans.fit_predict(X_scaled)

# Evaluate clustering performance
dbi = davies_bouldin_score(X_scaled, y_pred)
silhouette = silhouette_score(X_scaled, y_pred)
chs = calinski_harabasz_score(X_scaled, y_pred)

# Additional evaluation metrics for labeled data ( 'customer_satisfaction' is the label)
ari = adjusted_rand_score(df1['satisfaction'], y_pred)
jaccard = jaccard_score(df1['satisfaction'], y_pred, average='weighted')

print(f'Davies-Bouldin Index: {dbi}')
print(f'Silhouette Score: {silhouette}')
print(f'Calinski-Harabasz Score: {chs}')

print(f'Adjusted Rand Index (ARI): {ari}')
print(f'Jaccard Score: {jaccard}')


visualizing KMEANS

In [ ]:
from sklearn.decomposition import PCA

# Apply PCA to reduce dimensionality to 2
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

# Visualize the clustering results using a scatter plot
plt.figure(figsize=(10, 6))
sns.scatterplot(x=X_pca[:, 0], y=X_pca[:, 1], hue=y_pred, palette='viridis', legend='full')
plt.title('K-Means Clustering Results (PCA)')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.show()

In [ ]:
from sklearn.decomposition import PCA
import pandas as pd

# Assume X_scaled is your scaled data
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

# Get the principal components
components_df = pd.DataFrame(pca.components_, columns=X_scaled.columns)

# Display the principal components
print("Principal Components:")
print(components_df)


In [ ]:
from pandas.plotting import radviz
import matplotlib.pyplot as plt

# Add cluster labels to the DataFrame
df1['Cluster'] = y_pred

# Define a list of colors for each cluster
cluster_colors = ['red', 'green', 'blue']

# Increase the figure size
plt.figure(figsize=(27, 10))

# Visualize RadViz for each cluster separately
for cluster_num, color in zip(range(n_clusters), cluster_colors):
    plt.subplot(1, n_clusters, cluster_num + 1)
    cluster_data = df1[df1['Cluster'] == cluster_num]
    
    radviz(cluster_data, class_column='Cluster', color=color, alpha=0.5)
    plt.title(f'Cluster {cluster_num}')
    plt.xlabel('')
    plt.ylabel('')

plt.suptitle('RadViz - Clustering Visualization for Each Cluster', y=1.02)
plt.show()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Add cluster labels to the DataFrame
df1['Cluster'] = y_pred

# Increase the figure size
plt.figure(figsize=(15, 10))

# Create a heatmap
sns.heatmap(df1.groupby('Cluster').mean(), cmap='viridis', annot=True, fmt=".2f")
plt.title('Cluster Feature Means - Heatmap')
plt.show()


DBSCAN

neighborhood size

Elbow

In [ ]:
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

#  X_scaled is  standardized data
neigh = NearestNeighbors(n_neighbors=2)
nbrs = neigh.fit(X_scaled)
distances, indices = nbrs.kneighbors(X_scaled)

distances = np.sort(distances, axis=0)
distances = distances[:, 1]

plt.plot(distances)
plt.xlabel('Data Points')
plt.ylabel('Distance to 2nd Nearest Neighbor')
plt.title('Elbow Method for DBSCAN eps')
plt.show()


DBSCAN hyperparameters

In [ ]:
from sklearn.metrics import silhouette_score
from sklearn.cluster import DBSCAN


eps_values = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1,1.1,1.2,1,5]  
silhouette_scores = []

for eps in eps_values:
    dbscan = DBSCAN(eps=eps, min_samples=5)
    labels = dbscan.fit_predict(X_scaled)
    silhouette_scores.append(silhouette_score(X_scaled, labels))

best_eps = eps_values[np.argmax(silhouette_scores)]
print(f'Best eps value: {best_eps}')


Best eps value: 1

In [ ]:
from sklearn.cluster import DBSCAN
from sklearn.metrics import adjusted_rand_score

#  X_scaled is  standardized data and y is  labels
min_samples_values = [3, 5, 7, 10]  
eps = 1  

best_score = -1
best_min_samples = None

for min_samples in min_samples_values:
    dbscan = DBSCAN(eps=eps, min_samples=min_samples)
    labels = dbscan.fit_predict(X_scaled)
    score = adjusted_rand_score(y, labels)  #  y is  true labels
    print(f'Min Samples: {min_samples}, Adjusted Rand Score: {score}')

    if score > best_score:
        best_score = score
        best_min_samples = min_samples

print(f'Best Min Samples: {best_min_samples}, Best Adjusted Rand Score: {best_score}')


In [ ]:
X_scaled = df1.drop('satisfaction', axis=1)
y = df1['satisfaction']

from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score

metrics_to_try = ['euclidean', 'manhattan', 'cosine', 'minkowski', 'chebyshev']

best_metric = None
best_silhouette_score = -1

for metric in metrics_to_try:
    dbscan = DBSCAN(eps=1, min_samples=3, metric=metric)
    labels = dbscan.fit_predict(X_scaled)
    silhouette = silhouette_score(X_scaled, labels)
    
    print(f'Metric: {metric}, Silhouette Score: {silhouette}')
    
    if silhouette > best_silhouette_score:
        best_silhouette_score = silhouette
        best_metric = metric

print(f'Best Metric: {best_metric}, Best Silhouette Score: {best_silhouette_score}')



Best Metric: euclidean, Best Silhouette Score: 0.1348404334465667

DBSCAN model

In [ ]:
from sklearn.cluster import DBSCAN
from sklearn.metrics import davies_bouldin_score, silhouette_score, calinski_harabasz_score, adjusted_rand_score, jaccard_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns


## df1 is scaled df
X_scaled = df1.drop('satisfaction', axis=1)

# Set the parameters for DBSCAN
eps = 1
min_samples = 3  

# Initialize DBSCAN clustering
dbscan = DBSCAN(eps=eps, min_samples=min_samples, metric='euclidean')
labels = dbscan.fit_predict(X_scaled)

# Evaluate clustering performance
dbi = davies_bouldin_score(X_scaled, labels)
silhouette = silhouette_score(X_scaled, labels)
chs = calinski_harabasz_score(X_scaled, labels)
ari = adjusted_rand_score(df1['satisfaction'], labels)
jaccard = jaccard_score(df1['satisfaction'], labels, average='weighted')

# Print evaluation metrics
print(f'Davies-Bouldin Index: {dbi}')
print(f'Silhouette Score: {silhouette}')
print(f'Calinski-Harabasz Score: {chs}')
print(f'Adjusted Rand Index: {ari}')
print(f'Jaccard Score: {jaccard}')

# Visualize clusters using PCA
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

# Create a DataFrame with PCA components and cluster labels
df_pca = pd.DataFrame({'PCA1': X_pca[:, 0], 'PCA2': X_pca[:, 1], 'Cluster': labels})

# Plot the clusters
plt.figure(figsize=(10, 6))
sns.scatterplot(x='PCA1', y='PCA2', hue='Cluster', data=df_pca, palette='viridis', legend='full')
plt.title('DBSCAN Clusters (PCA)')
plt.show()


In [ ]:
noise_points_count = np.count_nonzero(labels == -1)
noise_points_count

In [ ]:
df

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Add cluster labels to the DataFrame
df1['Cluster'] = y_pred

# Increase the figure size
plt.figure(figsize=(15, 10))

# Create a heatmap
sns.heatmap(df1.groupby('Cluster').mean(), cmap='coolwarm', annot=True, fmt=".2f")
plt.title('Cluster Feature Means - Heatmap')
plt.show()


In [ ]:
from pandas.plotting import radviz
import matplotlib.pyplot as plt

# Add cluster labels to the DataFrame
df1['Cluster'] = y_pred

# Define a list of colors for each cluster
cluster_colors = ['purple', 'orange', 'gray']

# Increase the figure size
plt.figure(figsize=(27, 10))

# Adjust font size for better readability
plt.rcParams['font.size'] = 14

# Visualize RadViz for each cluster separately
for cluster_num, color in zip(range(n_clusters), cluster_colors):
    plt.subplot(1, n_clusters, cluster_num + 1)
    cluster_data = df1[df1['Cluster'] == cluster_num]
    
    radviz(cluster_data, class_column='Cluster', color=color, alpha=0.3)  # Reduce transparency
    plt.title(f'Cluster {cluster_num}')
    plt.xlabel('')
    plt.ylabel('')

# Adjust the distance between subplots
plt.subplots_adjust(wspace=0.5)

plt.suptitle('RadViz - Clustering Visualization for Each Cluster', y=1.2, fontsize=16)  # Adjust title font size
plt.show()


Davies-Bouldin Index: 2.6003055701454496
Silhouette Score: 0.1348404334465667
Calinski-Harabasz Score: 8155.810398563863
Adjusted Rand Index: 0.09935302975746158
Jaccard Score: 0.1590090402579417

HDBSCAN

In [ ]:
import numpy as np
from sklearn.metrics import silhouette_score
import hdbscan
from sklearn.model_selection import ParameterGrid
from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV



# Define the parameter grid for random search
param_dist = {
    'min_cluster_size': [5, 10, 15],
    'min_samples': [2, 3, 5, 7],
}

# Define silhouette scorer for optimization
silhouette_scorer = make_scorer(silhouette_score)

# Initialize HDBSCAN
hdb = hdbscan.HDBSCAN()

# Perform random search
random_search = RandomizedSearchCV(hdb, param_distributions=param_dist, scoring=silhouette_scorer, n_iter=50, cv=5)
random_search.fit(X_scaled)

# Get the best hyperparameters
best_params = random_search.best_params_

best_params

HDBSCAN model

In [ ]:
# Initialize HDBSCAN clustering
hdb = hdbscan.HDBSCAN(min_cluster_size=5, min_samples=2)
y_pred = hdb.fit_predict(X_scaled)

# Evaluate clustering performance
try:
    dbi = davies_bouldin_score(X_scaled, y_pred)
except ValueError:
    dbi = float('nan')

silhouette = silhouette_score(X_scaled, y_pred)
chs = calinski_harabasz_score(X_scaled, y_pred)

# Additional evaluation metrics for labeled data ( 'y' is the label)
ari = adjusted_rand_score(y, y_pred)
jaccard = jaccard_score(y, y_pred, average='weighted')

# Print the evaluation metrics
print(f'Davies-Bouldin Index: {dbi}')
print(f'Silhouette Score: {silhouette}')
print(f'Calinski-Harabasz Score: {chs}')
print(f'Adjusted Rand Index (ARI): {ari}')
print(f'Jaccard Score: {jaccard}')


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import hdbscan
from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler

#  X_scaled is  feature matrix
hdbscan_model = hdbscan.HDBSCAN(min_cluster_size=5, min_samples=2)  
labels = hdbscan_model.fit_predict(X_scaled)

# Apply PCA to reduce dimensionality
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

# Create a scatter plot with different colors for each cluster
unique_labels = set(labels)
colors = [plt.cm.Spectral(each) for each in np.linspace(0, 1, len(unique_labels))]

plt.figure(figsize=(10, 6))
for i, color in zip(unique_labels, colors):
    class_member_mask = (labels == i)
    xy = X_pca[class_member_mask]
    plt.scatter(xy[:, 0], xy[:, 1], c=[color], edgecolor='k', s=50, alpha=0.7)

plt.title('HDBSCAN Clustering with PCA')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.show()


hierarchical clustering (unfortunately Kernel Crashed)

hyperparam

In [ ]:
from scipy.cluster.hierarchy import dendrogram, linkage
import matplotlib.pyplot as plt

#  X is  feature matrix and Z is the linkage matrix
X = df1.drop('satisfaction', axis=1)

Z = linkage(X, method='ward')
plt.figure(figsize=(12, 6))
dendrogram(Z)
plt.title('Hierarchical Clustering Dendrogram')
plt.xlabel('Data Points')
plt.ylabel('Distance')
plt.show()


hierarchical model

In [ ]:
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import silhouette_score, adjusted_rand_score, jaccard_score
from scipy.cluster.hierarchy import dendrogram, linkage


#  X is  feature matrix
X = df1.drop('satisfaction', axis=1)

# Perform hierarchical clustering
Z = linkage(X, method='ward')

# Set the number of clusters (you can choose based on dendrogram)
n_clusters = 3

# Fit AgglomerativeClustering model
model = AgglomerativeClustering(n_clusters=n_clusters, linkage='ward')
labels = model.fit_predict(X)

# Evaluate clustering performance
silhouette = silhouette_score(X, labels)
ari = adjusted_rand_score(df1['satisfaction'], labels)
jaccard = jaccard_score(df1['satisfaction'], labels, average='weighted')

# Print evaluation metrics
print(f'Silhouette Score: {silhouette}')
print(f'Adjusted Rand Index: {ari}')
print(f'Jaccard Score: {jaccard}')

# Visualize dendrogram
plt.figure(figsize=(12, 6))
dendrogram(Z)
plt.title('Hierarchical Clustering Dendrogram')
plt.xlabel('Data Points')
plt.ylabel('Distance')
plt.show()


MEAN shift

hyperparam

In [ ]:
import pandas as pd
from sklearn.cluster import MeanShift, estimate_bandwidth
from sklearn.metrics import silhouette_score, make_scorer
from sklearn.model_selection import GridSearchCV

## df1 is scaled df
X_scaled = df1.drop('satisfaction', axis=1)

# Estimate bandwidth using sklearn's function
bandwidth = estimate_bandwidth(X_scaled, quantile=0.2, n_samples=500)

# Initialize Mean Shift clustering
meanshift = MeanShift(bandwidth=bandwidth)

# Define parameter grid for bandwidth
param_grid = {'bandwidth': [0.1, 0.5, 1.0]}

# Define a custom silhouette scorer
silhouette_scorer = make_scorer(silhouette_score)

# Perform GridSearchCV using the custom silhouette scorer
grid_search = GridSearchCV(meanshift, param_grid=param_grid, cv=5, scoring=silhouette_scorer, n_jobs=-1)
grid_search.fit(X_scaled)

# Get the best parameters
best_bandwidth = grid_search.best_params_['bandwidth']

# Initialize Mean Shift clustering with the best bandwidth
best_meanshift = MeanShift(bandwidth=best_bandwidth)
y_pred_best = best_meanshift.fit_predict(X_scaled)

# Evaluate clustering performance
silhouette_best = silhouette_score(X_scaled, y_pred_best)

print(f'Best Bandwidth: {best_bandwidth}')
print(f'Silhouette Score: {silhouette_best}')


high runtime causes using another method based on Sklearn

Meanshift Model

In [ ]:
import pandas as pd
from sklearn.cluster import MeanShift
from sklearn.metrics import davies_bouldin_score, silhouette_score, calinski_harabasz_score, adjusted_rand_score, jaccard_score
from sklearn.preprocessing import StandardScaler

# The following bandwidth can be automatically detected using
# bandwidth = estimate_bandwidth(X, quantile=0.2, n_samples=500)

## df1 is scaled df
X_scaled = df1.drop('satisfaction', axis=1)

# Initialize Mean Shift clustering
meanshift = MeanShift(bandwidth=0.1)
y_pred = meanshift.fit_predict(X_scaled)

# Evaluate clustering performance
dbi = davies_bouldin_score(X_scaled, y_pred)
silhouette = silhouette_score(X_scaled, y_pred)
chs = calinski_harabasz_score(X_scaled, y_pred)

# Additional evaluation metrics for labeled data
ari = adjusted_rand_score(y, y_pred)
jaccard = jaccard_score(y, y_pred, average='weighted')

print(f'Davies-Bouldin Index: {dbi}')
print(f'Silhouette Score: {silhouette}')
print(f'Calinski-Harabasz Score: {chs}')

print(f'Adjusted Rand Index (ARI): {ari}')
print(f'Jaccard Score: {jaccard}')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import MeanShift
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

#  X_scaled is  feature matrix
meanshift = MeanShift(bandwidth=0.1)
labels = meanshift.fit_predict(X_scaled)

# Apply PCA to reduce dimensionality
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

# Create a scatter plot with different colors for each cluster
unique_labels = set(labels)
colors = [plt.cm.Spectral(each) for each in np.linspace(0, 1, len(unique_labels))]

plt.figure(figsize=(10, 6))
for i, color in zip(unique_labels, colors):
    class_member_mask = (labels == i)
    xy = X_pca[class_member_mask, :]
    plt.scatter(xy[:, 0], xy[:, 1], c=[color], edgecolor='k', s=50, alpha=0.7)

plt.title('Mean Shift Clustering with PCA')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.show()


Gaussion MM

hyperparam GMM Method1

In [ ]:
from sklearn.mixture import GaussianMixture

#  X is  feature matrix
X = df1.drop('satisfaction', axis=1)

# Define the range of n_components to search over
n_components_range = range(2, 6)

# Perform a grid search
best_n_components = None
best_aic = float('inf')  # Initialize with a large value
for n_components in n_components_range:
    gmm = GaussianMixture(n_components=n_components)
    gmm.fit(X)
    aic = gmm.aic(X)
    if aic < best_aic:
        best_aic = aic
        best_n_components = n_components

print(f"Best n_components: {best_n_components}")


GMM model based on n_component=5

In [ ]:
# Set the number of clusters for GMM
n_components = 5


## df1 is scaled df
X_scaled = df1.drop('satisfaction', axis=1)

# Initialize Gaussian Mixture Model clustering
gmm = GaussianMixture(n_components=n_components, random_state=42)
y_pred = gmm.fit_predict(X_scaled)

# Evaluate clustering performance
dbi = davies_bouldin_score(X_scaled, y_pred)
silhouette = silhouette_score(X_scaled, y_pred)
chs = calinski_harabasz_score(X_scaled, y_pred)

# Additional evaluation metrics for labeled data ( 'satisfaction' is the label)
ari = adjusted_rand_score(df1['satisfaction'], y_pred)
jaccard = jaccard_score(df1['satisfaction'], y_pred, average='weighted')

print(f'Davies-Bouldin Index: {dbi}')
print(f'Silhouette Score: {silhouette}')
print(f'Calinski-Harabasz Score: {chs}')

print(f'Adjusted Rand Index (ARI): {ari}')
print(f'Jaccard Score: {jaccard}')


hyperparam GMM Method2

In [ ]:
from sklearn.metrics import silhouette_score
from sklearn.mixture import GaussianMixture

#  X is  feature matrix
X = df1.drop('satisfaction', axis=1)

# Define the range of n_components to search over
n_components_range = range(2, 6)

# Evaluate silhouette score for each n_components
silhouette_scores = []
for n_components in n_components_range:
    gmm = GaussianMixture(n_components=n_components)
    labels = gmm.fit_predict(X)
    silhouette_scores.append(silhouette_score(X, labels))

best_n_components = n_components_range[np.argmax(silhouette_scores)]
print(f"Best n_components (Silhouette Score): {best_n_components}")


it is better to select number of component parallel with covariance type for being more accurate:

In [ ]:
import pandas as pd
from sklearn.mixture import GaussianMixture
from sklearn.metrics import davies_bouldin_score, silhouette_score, calinski_harabasz_score, adjusted_rand_score, jaccard_score

# Set the hyperparameters to tune
param_grid = {
    'n_components': [2, 3, 4, 5], 
    'covariance_type': ['full', 'tied', 'diag', 'spherical'],
}

# Initialize Gaussian Mixture Model
gmm = GaussianMixture(random_state=42)

# Define a custom silhouette scorer
silhouette_scorer = make_scorer(silhouette_score)

# Initialize GridSearchCV
grid_search = GridSearchCV(gmm, param_grid=param_grid, cv=5, scoring=silhouette_scorer, n_jobs=-1)

# Fit the model to find the best hyperparameters
grid_search.fit(X_scaled)

# Get the best hyperparameters
best_params = grid_search.best_params_
best_gmm = grid_search.best_estimator_

# Print the best hyperparameters
print(f'Best Hyperparameters: {best_params}')

GMM model based on Best Hyperparameters: {'covariance_type': 'full', 'n_components': 2}

In [ ]:
# Set the number of clusters for GMM
n_components = 2
covariance_type = 'full'

## df1 is scaled df
X_scaled = df1.drop('satisfaction', axis=1)

# Initialize Gaussian Mixture Model clustering
gmm = GaussianMixture(n_components=n_components, covariance_type=covariance_type, random_state=42)
y_pred = gmm.fit_predict(X_scaled)

# Evaluate clustering performance
dbi = davies_bouldin_score(X_scaled, y_pred)
silhouette = silhouette_score(X_scaled, y_pred)
chs = calinski_harabasz_score(X_scaled, y_pred)

# Additional evaluation metrics for labeled data ( 'satisfaction' is the label)
ari = adjusted_rand_score(df1['satisfaction'], y_pred)
jaccard = jaccard_score(df1['satisfaction'], y_pred, average='weighted')

print(f'Davies-Bouldin Index: {dbi}')
print(f'Silhouette Score: {silhouette}')
print(f'Calinski-Harabasz Score: {chs}')

print(f'Adjusted Rand Index (ARI): {ari}')
print(f'Jaccard Score: {jaccard}')


OPTICS

In [ ]:
import pandas as pd
from sklearn.cluster import OPTICS
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

# Standardize the data
y = df1['satisfaction']
X_scaled = df1.drop('satisfaction', axis=1)

# Define the parameter grid
param_grid = {
    'min_samples': [3, 5, 7],
    'xi': [0.01, 0.05, 0.1]
}

# Initialize OPTICS clustering
optics = OPTICS()

# Define a custom silhouette scorer
silhouette_scorer = make_scorer(silhouette_score)

# Initialize GridSearchCV
grid_search = GridSearchCV(optics, param_grid=param_grid, cv=5, scoring=silhouette_scorer, n_jobs=-1)

# Fit the model to find the best hyperparameters
grid_search.fit(X_scaled)

# Get the best hyperparameters
best_params = grid_search.best_params_

# Print the best hyperparameters
print(f'Best Hyperparameters: {best_params}')


elbow method for min_cluster

In [ ]:
from sklearn.cluster import OPTICS
import matplotlib.pyplot as plt

optics = OPTICS(xi=0.01)
optics.fit(X_scaled)

reachability = optics.reachability_[optics.ordering_]
plt.plot(reachability)
plt.title("OPTICS Reachability Plot")
plt.xlabel("Ordered Data Points")
plt.ylabel("Reachability Distance")
plt.show()


There are two basic ways to extract clusters from the reachability plot – manual and automatic using different algorithms. The manual way refers to either setting the range on the x-axis or using the threshold on the y-axis after performing the visual inspection of the reachability plot. Generally, clusters appear as valleys on the reachability plot so that deeper valleys represent dense clusters, while shallow valleys represent sparse clusters.

 The reachability plot showing the ordering returned by the OPTICS algorithm visibly has 3 valleys corresponding to the three identified clusters. We can see that the densest cluster

OPTICS model

In [ ]:
from sklearn.cluster import OPTICS
from sklearn.metrics import davies_bouldin_score, silhouette_score, calinski_harabasz_score, adjusted_rand_score, jaccard_score

# Standardize the data
y = df1['satisfaction']
X_scaled = df1.drop('satisfaction', axis=1)

# Initialize OPTICS clustering
optics = OPTICS(xi=0.01, min_cluster_size=3)

# Fit the model on the entire dataset
y_pred = optics.fit_predict(X_scaled)

# Evaluate clustering performance
dbi = davies_bouldin_score(X_scaled, y_pred)
silhouette = silhouette_score(X_scaled, y_pred)
chs = calinski_harabasz_score(X_scaled, y_pred)

# Additional evaluation metrics for labeled data
ari = adjusted_rand_score(y, y_pred)
jaccard = jaccard_score(y, y_pred, average='weighted')

print(f'Davies-Bouldin Index: {dbi}')
print(f'Silhouette Score: {silhouette}')
print(f'Calinski-Harabasz Score: {chs}')
print(f'Adjusted Rand Index (ARI): {ari}')
print(f'Jaccard Score: {jaccard}')


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import OPTICS
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

#  X_scaled is  feature matrix
optics = OPTICS(min_samples=3, xi=0.01, min_cluster_size=3) 
labels = optics.fit_predict(X_scaled)

# Apply PCA to reduce dimensionality
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

# Create a scatter plot with different colors for each cluster
unique_labels = set(labels)
colors = [plt.cm.Spectral(each) for each in np.linspace(0, 1, len(unique_labels))]

plt.figure(figsize=(10, 6))
for i, color in zip(unique_labels, colors):
    class_member_mask = (labels == i)
    xy = X_pca[class_member_mask, :]
    plt.scatter(xy[:, 0], xy[:, 1], c=[color], edgecolor='k', s=50, alpha=0.7)

plt.title('OPTICS Clustering with PCA')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.show()


Creating features

In [ ]:
df3

In [ ]:
X = df3.drop('satisfaction', axis=1)

In [ ]:
X['Delays'] = X['Departure Delay in Minutes'] + X['Departure/Arrival time convenient']


In [ ]:
X['DPD'] = X['Delays']/X['Flight Distance']
X['D*D'] = X['Delays']*X['Flight Distance']


In [ ]:
X

In [ ]:
X1 = X[['DPD','Total_Score','Age']]
X2 = X[['D*D','Total_Score','Age']]

In [ ]:
X1

In [ ]:
#scalling process specified for X1
c = X1.columns

from sklearn.preprocessing import MinMaxScaler

Scaler = MinMaxScaler()
X1 = Scaler.fit_transform(X1)

X1 = pd.DataFrame(X1)

X1.columns = c

In [ ]:
X1

K-means

In [ ]:
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt


distortions = []
K_range = range(2, 9)

for k in K_range:
    kmeanModel = KMeans(n_clusters=k)
    kmeanModel.fit(X1)
    distortions.append(kmeanModel.inertia_)

plt.figure(figsize=(8, 6))
plt.plot(K_range, distortions, marker='o')
plt.title('Elbow Method for Optimal k')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Distortion (SSD)')
plt.show()


In [ ]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.metrics import davies_bouldin_score, silhouette_score, calinski_harabasz_score, adjusted_rand_score, jaccard_score
from sklearn.preprocessing import StandardScaler

## df1 is scaled df
X_scaled = X1
y = df1['satisfaction']

# Set the number of clusters for k-means
n_clusters = 3

# Initialize k-means clustering with K-means++ initialization
kmeans = KMeans(n_clusters=n_clusters, init='k-means++', random_state=42)
y_pred = kmeans.fit_predict(X_scaled)

# Evaluate clustering performance
dbi = davies_bouldin_score(X_scaled, y_pred)
silhouette = silhouette_score(X_scaled, y_pred)
chs = calinski_harabasz_score(X_scaled, y_pred)

# Additional evaluation metrics for labeled data ( 'customer_satisfaction' is the label)
ari = adjusted_rand_score(df1['satisfaction'], y_pred)
jaccard = jaccard_score(df1['satisfaction'], y_pred, average='weighted')

print(f'Davies-Bouldin Index: {dbi}')
print(f'Silhouette Score: {silhouette}')
print(f'Calinski-Harabasz Score: {chs}')

print(f'Adjusted Rand Index (ARI): {ari}')
print(f'Jaccard Score: {jaccard}')


In [ ]:
# Count the number of data points in each cluster
cluster_counts = pd.Series(y_pred).value_counts().sort_index()
print('Number of data points in each cluster:')
print(cluster_counts)

In [ ]:
# Convert Pandas DataFrame to NumPy array
X_np = X_scaled.values

# Create a 3D scatter plot with Plotly Express
fig = px.scatter_3d(
    x=X_np[:, 0],
    y=X_np[:, 1],
    z=X_np[:, 2],
    color=y_pred,
    labels={'color': 'Cluster'},
    title='3D Clustering Visualization (K-Means)',
    opacity=0.7,           # Set opacity for better visibility of overlapping points
    size_max=15,           # Adjust the maximum marker size
    template='plotly_dark',  # Choose a plotly template (e.g., 'plotly', 'plotly_dark', 'plotly_white')
    color_continuous_scale='Viridis',  # Choose a color scale (e.g., 'Viridis', 'Jet', 'Plasma')
    width=800,             # Set plot width
    height=600,            # Set plot height
)

# Customize the layout (optional)
fig.update_layout(
    scene=dict(
        xaxis_title='DPD',
        yaxis_title='Total_Score',
        zaxis_title='Age',
        aspectmode='cube',  # Set aspect mode for equal scaling
        camera=dict(
            eye=dict(x=1.5, y=1.5, z=1.5)  # Adjust the camera position for better viewing
        )  
    ),
)

# Show the plot
fig.show()

In [ ]:
# Convert Pandas DataFrame to NumPy array
X_np = X_scaled.values

# Create a 3D scatter plot with Plotly Express
fig = px.scatter_3d(
    x=X_np[:, 0],
    y=X_np[:, 1],
    z=X_np[:, 2],
    color=y_pred,
    labels={'color': 'Cluster'},
    title='3D Clustering Visualization (K-Means)',
    opacity=0.7,           # Set opacity for better visibility of overlapping points
    size_max=15,           # Adjust the maximum marker size
    template='plotly_dark',  # Choose a plotly template (e.g., 'plotly', 'plotly_dark', 'plotly_white')
    color_continuous_scale='Viridis',  # Choose a color scale (e.g., 'Viridis', 'Jet', 'Plasma')
    width=800,             # Set plot width
    height=600,            # Set plot height
)

# Customize the layout (optional)
fig.update_layout(
    scene=dict(
        xaxis_title='D*D',
        yaxis_title='Total_Score',
        zaxis_title='Age',
        aspectmode='cube',  # Set aspect mode for equal scaling
        camera=dict(
            eye=dict(x=1.5, y=1.5, z=1.5)  # Adjust the camera position for better viewing
        )  
    ),
)

# Show the plot
fig.show()

DBSCAN

In [ ]:
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

#  X_scaled is  standardized data
X_scaled = X1
neigh = NearestNeighbors(n_neighbors=2)
nbrs = neigh.fit(X_scaled)
distances, indices = nbrs.kneighbors(X_scaled)

distances = np.sort(distances, axis=0)
distances = distances[:, 1]

plt.plot(distances)
plt.xlabel('Data Points')
plt.ylabel('Distance to 2nd Nearest Neighbor')
plt.title('Elbow Method for DBSCAN eps')
plt.show()


In [ ]:
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
import numpy as np

X_scaled = X1
y = df1['satisfaction']  #  'satisfaction' is the target variable

# Varying values for eps and min_samples
eps_values = [0.05 , 0.06 , 0.07 , 0.08 , 0.09 , 0.1]
min_samples_values = [2, 3, 4]

best_eps = None
best_min_samples = None
best_silhouette_score = -1

# Iterate through parameter combinations
for eps in eps_values:
    for min_samples in min_samples_values:
        # Initialize DBSCAN clustering
        dbscan = DBSCAN(eps=eps, min_samples=min_samples)
        labels = dbscan.fit_predict(X_scaled)
        
        # Evaluate clustering performance using silhouette score
        silhouette = silhouette_score(X_scaled, labels)
        
        # Print silhouette score for each parameter combination
        print(f'eps={eps}, min_samples={min_samples}, Silhouette Score: {silhouette}')
        
        # Update best parameters if a higher silhouette score is achieved
        if silhouette > best_silhouette_score:
            best_eps = eps
            best_min_samples = min_samples
            best_silhouette_score = silhouette

# Print the best parameters
print(f'Best Parameters: eps={best_eps}, min_samples={best_min_samples}, Best Silhouette Score: {best_silhouette_score}')

# Apply DBSCAN with the best parameters
best_dbscan = DBSCAN(eps=best_eps, min_samples=best_min_samples)
best_labels = best_dbscan.fit_predict(X_scaled)

eps=0.01, min_samples=2, Silhouette Score: 0.438025828350863
eps=0.01, min_samples=3, Silhouette Score: 0.41871060744020394
eps=0.01, min_samples=4, Silhouette Score: 0.4077674735204568
eps=0.02, min_samples=2, Silhouette Score: -0.5953789357856237
eps=0.02, min_samples=3, Silhouette Score: -0.5745465962869623
eps=0.02, min_samples=4, Silhouette Score: -0.5650357410041018
eps=0.03, min_samples=2, Silhouette Score: -0.36541525484261
eps=0.03, min_samples=3, Silhouette Score: -0.3012610314189669
eps=0.03, min_samples=4, Silhouette Score: -0.20222824767499073
eps=0.04, min_samples=2, Silhouette Score: -0.16010032060326204
eps=0.04, min_samples=3, Silhouette Score: -0.030513750415238346
eps=0.04, min_samples=4, Silhouette Score: 0.06319758079191402
eps=0.05, min_samples=2, Silhouette Score: 0.037731286410677786
eps=0.05, min_samples=3, Silhouette Score: 0.1044823054291336
eps=0.05, min_samples=4, Silhouette Score: 0.19523143620025368
Best Parameters: eps=0.01, min_samples=2, Best Silhouette Score: 0.438025828350863

In [ ]:
from sklearn.cluster import DBSCAN
from sklearn.metrics import davies_bouldin_score
from sklearn.preprocessing import StandardScaler
import numpy as np


X_scaled = X1
y = df1['satisfaction']  #  'satisfaction' is the target variable

# Varying values for eps and min_samples
eps_values = [0.01, 0.02, 0.03, 0.04, 0.05]
min_samples_values = [2, 3, 4, 5, 6]

best_eps = None
best_min_samples = None
best_dbi = float('inf')  # Initialize to a high value

# Iterate through parameter combinations
for eps in eps_values:
    for min_samples in min_samples_values:
        # Initialize DBSCAN clustering
        dbscan = DBSCAN(eps=eps, min_samples=min_samples)
        labels = dbscan.fit_predict(X_scaled)
        
        # Evaluate clustering performance using Davies-Bouldin Index
        dbi = davies_bouldin_score(X_scaled, labels)
        
        # Print Davies-Bouldin Index for each parameter combination
        print(f'eps={eps}, min_samples={min_samples}, Davies-Bouldin Index: {dbi}')
        
        # Update best parameters if a lower Davies-Bouldin Index is achieved
        if dbi < best_dbi:
            best_eps = eps
            best_min_samples = min_samples
            best_dbi = dbi

# Print the best parameters
print(f'Best Parameters: eps={best_eps}, min_samples={best_min_samples}, Best Davies-Bouldin Index: {best_dbi}')

# Apply DBSCAN with the best parameters
best_dbscan = DBSCAN(eps=best_eps, min_samples=best_min_samples)
best_labels = best_dbscan.fit_predict(X_scaled)

# Visualize or further analyze the clusters obtained with the best parameters


In [ ]:
from sklearn.cluster import DBSCAN
from sklearn.metrics import adjusted_rand_score

#  X1 is  DataFrame with the selected columns
X_scaled = X1
y = df1['satisfaction']  #  'satisfaction' is the target variable

min_samples_values = [3, 5, 7, 10]  # Adjustting the range based on data
eps = 0.025  # Adjust the epsilon value

best_score = -1
best_min_samples = None

for min_samples in min_samples_values:
    dbscan = DBSCAN(eps=eps, min_samples=min_samples)
    labels = dbscan.fit_predict(X_scaled)
    score = adjusted_rand_score(y, labels)
    print(f'Min Samples: {min_samples}, Adjusted Rand Score: {score}')

    if score > best_score:
        best_score = score
        best_min_samples = min_samples

print(f'Best Min Samples: {best_min_samples}, Best Adjusted Rand Score: {best_score}')


In [ ]:
from sklearn.cluster import DBSCAN
from sklearn.metrics import davies_bouldin_score, silhouette_score, calinski_harabasz_score, adjusted_rand_score, jaccard_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns

# Setting the parameters for DBSCAN
eps = 0.025  # Adjust based on  data
min_samples = 3  # Adjust based on  data

# Initialize DBSCAN clustering
dbscan = DBSCAN(eps=eps, min_samples=min_samples)
labels = dbscan.fit_predict(X_scaled)

# Evaluate clustering performance
dbi = davies_bouldin_score(X_scaled, labels)
silhouette = silhouette_score(X_scaled, labels)
chs = calinski_harabasz_score(X_scaled, labels)
ari = adjusted_rand_score(df1['satisfaction'], labels)
jaccard = jaccard_score(df1['satisfaction'], labels, average='weighted')

# Print evaluation metrics
print(f'Davies-Bouldin Index: {dbi}')
print(f'Silhouette Score: {silhouette}')
print(f'Calinski-Harabasz Score: {chs}')
print(f'Adjusted Rand Index: {ari}')
print(f'Jaccard Score: {jaccard}')



In [ ]:
#  'best_labels' is the cluster labels obtained from DBSCAN
unique_labels, counts = np.unique(best_labels, return_counts=True)

# Display counts for each class
for label, count in zip(labels, counts):
    print(f'Class {label}: {count} points')

Hierarchical (Kernel crashed)

In [ ]:
from scipy.cluster.hierarchy import dendrogram, linkage
import matplotlib.pyplot as plt

#  X is  feature matrix and Z is the linkage matrix
X = X1 [['DPD','Total_Score']]

Z = linkage(X, method='ward')
plt.figure(figsize=(12, 6))
dendrogram(Z)
plt.title('Hierarchical Clustering Dendrogram')
plt.xlabel('Data Points')
plt.ylabel('Distance')
plt.show()


In [ ]:
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import silhouette_score, adjusted_rand_score, jaccard_score
from scipy.cluster.hierarchy import dendrogram, linkage


#  X is  feature matrix
X = df1.drop('satisfaction', axis=1)

# Perform hierarchical clustering
Z = linkage(X, method='ward')

# Set the number of clusters (you can choose based on dendrogram)
n_clusters = 3

# Fit AgglomerativeClustering model
model = AgglomerativeClustering(n_clusters=n_clusters, linkage='ward')
labels = model.fit_predict(X)

# Evaluate clustering performance
silhouette = silhouette_score(X, labels)
ari = adjusted_rand_score(df1['satisfaction'], labels)
jaccard = jaccard_score(df1['satisfaction'], labels, average='weighted')

# Print evaluation metrics
print(f'Silhouette Score: {silhouette}')
print(f'Adjusted Rand Index: {ari}')
print(f'Jaccard Score: {jaccard}')

# Visualize dendrogram
plt.figure(figsize=(12, 6))
dendrogram(Z)
plt.title('Hierarchical Clustering Dendrogram')
plt.xlabel('Data Points')
plt.ylabel('Distance')
plt.show()


BRICH

In [ ]:
from sklearn.cluster import Birch
from sklearn.metrics import davies_bouldin_score
from sklearn.model_selection import ParameterGrid
import numpy as np

#  X1 is  DataFrame with the selected columns
X_scaled = X1
y = df1['satisfaction']  #  'satisfaction' is the target variable

# Define parameter grid for hyperparameter tuning
param_grid = {
    'threshold': [0.1, 0.2, 0.3, 0.4, 0.5],
    'branching_factor': [10, 20, 30, 40, 50]
}

# Generate all combinations of hyperparameters
param_combinations = list(ParameterGrid(param_grid))

best_params = None
best_dbi = float('inf')  # Initialize to a high value

# Iterate through parameter combinations
for params in param_combinations:
    # Initialize BIRCH clustering with current hyperparameters
    birch = Birch(**params)
    labels = birch.fit_predict(X_scaled)
    
    # Check if there are more than one unique label (cluster)
    unique_labels = np.unique(labels)
    if len(unique_labels) > 1:
        # Evaluate clustering performance using Davies-Bouldin Index
        dbi = davies_bouldin_score(X_scaled, labels)
        
        # Print evaluation metrics for each parameter combination
        print(f'Parameters: {params}')
        print(f'Davies-Bouldin Index: {dbi}')
        
        # Update best parameters if a lower Davies-Bouldin Index is achieved
        if dbi < best_dbi:
            best_params = params
            best_dbi = dbi

# Print the best parameters
print(f'Best Parameters: {best_params}, Best Davies-Bouldin Index: {best_dbi}')

# Apply BIRCH with the best parameters
best_birch = Birch(**best_params)
best_labels = best_birch.fit_predict(X_scaled)

# Visualize or further analyze the clusters obtained with the best parameters


In [ ]:
from sklearn.metrics import davies_bouldin_score, silhouette_score, calinski_harabasz_score, adjusted_rand_score, jaccard_score

#  'best_labels' is the cluster labels obtained from BIRCH

# Evaluate clustering performance using Davies-Bouldin Index, Silhouette Score, Calinski-Harabasz Score, Adjusted Rand Index, and Jaccard Score
dbi = davies_bouldin_score(X_scaled, best_labels)
silhouette = silhouette_score(X_scaled, best_labels)
chs = calinski_harabasz_score(X_scaled, best_labels)
ari = adjusted_rand_score(df1['satisfaction'], best_labels)
jaccard = jaccard_score(df1['satisfaction'], best_labels, average='weighted')

# Print evaluation metrics after fitting the model
print(f'Davies-Bouldin Index: {dbi}')
print(f'Silhouette Score: {silhouette}')
print(f'Calinski-Harabasz Score: {chs}')
print(f'Adjusted Rand Index: {ari}')
print(f'Jaccard Score: {jaccard}')


In [ ]:
import plotly.express as px

#  'best_labels' contains the cluster labels obtained from BIRCH or any clustering algorithm
#  'X_scaled' is  preprocessed and scaled data

# Convert Pandas DataFrame to NumPy array
X_np = X_scaled.values

# Create a 3D scatter plot with Plotly Express
fig = px.scatter_3d(
    x=X_np[:, 0],
    y=X_np[:, 1],
    z=X_np[:, 2],
    color=best_labels,
    labels={'color': 'Cluster'},
    title='3D Clustering Visualization',
    opacity=0.7,           # Set opacity for better visibility of overlapping points
    size_max=15,           # Adjust the maximum marker size
    template='plotly_dark',  # Choose a plotly template (e.g., 'plotly', 'plotly_dark', 'plotly_white')
    color_continuous_scale='Viridis',  # Choose a color scale (e.g., 'Viridis', 'Jet', 'Plasma')
    width=800,             # Set plot width
    height=600,            # Set plot height
)

# Customize the layout (optional)
fig.update_layout(
    scene=dict(
        xaxis_title='DPD',
        yaxis_title='Total_Score',
        zaxis_title='Age',
        aspectmode='cube',  # Set aspect mode for equal scaling
    ),
)

# Show the plot
fig.show()


 Fuzzy C-Means (FCM)

In [ ]:
import numpy as np
from sklearn.metrics import davies_bouldin_score
from skfuzzy.cluster import cmeans
from sklearn.model_selection import ParameterGrid

#  X1 is  DataFrame with the selected columns
X_scaled = X1.values
y = df1['satisfaction']  #  'satisfaction' is the target variable

# Define parameter grid for hyperparameter tuning
param_grid = {
    'c': [1, 2, 3, 4, 5],     # Number of clusters
    'm': [1.1, 1.5, 2.0, 2.5],  # Fuzziness parameter
    'error': [0.001, 0.01, 0.1, 0.5],  # Tolerance to stop iteration
    'maxiter': [50, 100, 150],  # Maximum number of iterations
}

# Generate all combinations of hyperparameters
param_combinations = list(ParameterGrid(param_grid))

best_params = None
best_dbi = float('inf')  # Initialize to a high value

# Iterate through parameter combinations
for params in param_combinations:
    # Initialize FCM clustering with current hyperparameters
    cntr, u, _, _, _, _, _ = cmeans(X_scaled.T, params['c'], params['m'], error=params['error'], maxiter=params['maxiter'])
    
    # Get cluster labels based on the highest membership values
    labels = np.argmax(u, axis=0)
    
    # Check if the number of unique labels is greater than 1
    if len(np.unique(labels)) <= 1:
        continue
    
    # Evaluate clustering performance using Davies-Bouldin Index
    dbi = davies_bouldin_score(X_scaled, labels)
    
    # Print evaluation metrics for each parameter combination
    print(f'Parameters: {params}')
    print(f'Davies-Bouldin Index: {dbi}')
    
    # Update best parameters if a lower Davies-Bouldin Index is achieved
    if dbi < best_dbi:
        best_params = params
        best_dbi = dbi

# Print the best parameters
print(f'Best Parameters: {best_params}, Best Davies-Bouldin Index: {best_dbi}')

# Apply FCM with the best parameters
best_cntr, best_u, _, _, _, _, _ = cmeans(X_scaled.T, best_params['c'], best_params['m'], error=best_params['error'], maxiter=best_params['maxiter'])
best_labels = np.argmax(best_u, axis=0)

# Visualize or further analyze the clusters obtained with the best parameters


In [ ]:
from sklearn.metrics import davies_bouldin_score, silhouette_score, calinski_harabasz_score, adjusted_rand_score, jaccard_score

#  'best_labels' is the cluster labels obtained from  Fuzzy C-Means (FCM)

# Evaluate clustering performance using Davies-Bouldin Index, Silhouette Score, Calinski-Harabasz Score, Adjusted Rand Index, and Jaccard Score
dbi = davies_bouldin_score(X_scaled, best_labels)
silhouette = silhouette_score(X_scaled, best_labels)
chs = calinski_harabasz_score(X_scaled, best_labels)
ari = adjusted_rand_score(df1['satisfaction'], best_labels)
jaccard = jaccard_score(df1['satisfaction'], best_labels, average='weighted')

# Print evaluation metrics after fitting the model
print(f'Davies-Bouldin Index: {dbi}')
print(f'Silhouette Score: {silhouette}')
print(f'Calinski-Harabasz Score: {chs}')
print(f'Adjusted Rand Index: {ari}')
print(f'Jaccard Score: {jaccard}')


In [ ]:
import plotly.express as px

# Create a 3D scatter plot with Plotly Express
fig = px.scatter_3d(
    x=X_np[:, 0],
    y=X_np[:, 1],
    z=X_np[:, 2],
    color=best_labels,
    labels={'color': 'Cluster'},
    title='3D Clustering Visualization',
    opacity=0.7,           # Set opacity for better visibility of overlapping points
    size_max=15,           # Adjust the maximum marker size
    template='plotly_dark',  # Choose a Plotly template (e.g., 'plotly', 'plotly_dark', 'plotly_white')
    color_continuous_scale='Viridis',  # Choose a color scale (e.g., 'Viridis', 'Jet', 'Plasma')
    width=800,             # Set plot width
    height=600,            # Set plot height
)

# Customize the layout (optional)
fig.update_layout(
    scene=dict(
        xaxis_title='DPD',
        yaxis_title='Total_Score',
        zaxis_title='Age',
        aspectmode='cube',  # Set aspect mode for equal scaling
    ),
)

# Show the plot
fig.show()
